In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 전체 코드

In [ ]:
# ====== Cell 1: 공통 import & 경로 설정 ======
import os
import json
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional, Set

import numpy as np
from tqdm import tqdm

# Colab에서 Spider 전처리 결과가 있는 디렉토리
PREPROCESSED_DIR = "/content/drive/MyDrive/ai_intensive2/spider_data/preprocessed"

TABLE_INDEX_JSONL = os.path.join(PREPROCESSED_DIR, "table_index.jsonl")
TABLE_INDEX_EMB   = os.path.join(PREPROCESSED_DIR, "table_index_emb.npy")

SCHEMA_INDEX_JSONL = os.path.join(PREPROCESSED_DIR, "schema_index.jsonl")
SCHEMA_INDEX_EMB   = os.path.join(PREPROCESSED_DIR, "schema_index_emb.npy")

CELL_INDEX_JSONL = os.path.join(PREPROCESSED_DIR, "cell_index.jsonl")
CELL_INDEX_EMB   = os.path.join(PREPROCESSED_DIR, "cell_index_emb.npy")

print("Preprocessed dir:", PREPROCESSED_DIR)


Preprocessed dir: /content/drive/MyDrive/ai_intensive2/spider_data/preprocessed


In [ ]:
# ====== Cell 2: 임베딩 모델 로딩 (BGE-M3, 1024차원) ======
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer

# ⚠️ 오프라인 인덱스와 차원을 맞추기 위해 BGE-M3 사용 (1024-dim)
embedding_model = SentenceTransformer("BAAI/bge-m3")

def embed_texts(texts: List[str]) -> np.ndarray:
    """
    BGE-M3 임베딩 + L2 정규화.
    """
    emb = embedding_model.encode(texts, normalize_embeddings=True)
    return np.asarray(emb, dtype=np.float32)

print("Embedding model loaded. dim =", embedding_model.get_sentence_embedding_dimension())


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Embedding model loaded. dim = 1024


In [ ]:
# ====== Cell 3: 0단계 - 인덱스 로더 정의 ======

@dataclass
class TableIndexRow:
    db_id: str
    table_idx: int
    table_name_norm: str
    table_name_db: str
    columns: List[str]
    text: str

class TableIndexDB:
    def __init__(self, rows: List[TableIndexRow], emb: np.ndarray):
        self.rows = rows
        self.emb = emb  # shape [N, dim]

def load_table_index_db(
    jsonl_path: str = TABLE_INDEX_JSONL,
    emb_path: str   = TABLE_INDEX_EMB,
) -> TableIndexDB:
    rows: List[TableIndexRow] = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            rows.append(
                TableIndexRow(
                    db_id=obj["db_id"],
                    table_idx=int(obj["table_idx"]),
                    table_name_norm=obj["table_name_norm"],
                    table_name_db=obj["table_name_db"],
                    columns=list(obj["columns"]),
                    text=obj["text"],
                )
            )
    emb = np.load(emb_path)
    assert len(rows) == emb.shape[0], f"table_index: rows({len(rows)}) != emb({emb.shape[0]})"
    print(f"[TableIndexDB] Loaded {len(rows)} rows, emb shape={emb.shape}")
    return TableIndexDB(rows, emb)

@dataclass
class SchemaIndexRow:
    db_id: str
    table_idx: int
    table_name_norm: str
    table_name_db: str
    col_id: int
    col_name_norm: str
    col_name_db: str
    col_type: str
    is_pk: bool
    is_fk: bool
    text: str

class SchemaIndexDB:
    def __init__(self, rows: List[SchemaIndexRow], emb: np.ndarray):
        self.rows = rows
        self.emb = emb  # shape [N, dim]

def load_schema_index_db(
    jsonl_path: str = SCHEMA_INDEX_JSONL,
    emb_path: str   = SCHEMA_INDEX_EMB,
) -> SchemaIndexDB:
    rows: List[SchemaIndexRow] = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            rows.append(
                SchemaIndexRow(
                    db_id=obj["db_id"],
                    table_idx=int(obj["table_idx"]),
                    table_name_norm=obj["table_name_norm"],
                    table_name_db=obj["table_name_db"],
                    col_id=int(obj["col_id"]),
                    col_name_norm=obj["col_name_norm"],
                    col_name_db=obj["col_name_db"],
                    col_type=obj["col_type"],
                    is_pk=bool(obj["is_pk"]),
                    is_fk=bool(obj["is_fk"]),
                    text=obj["text"],
                )
            )
    emb = np.load(emb_path)
    assert len(rows) == emb.shape[0], f"schema_index: rows({len(rows)}) != emb({emb.shape[0]})"
    print(f"[SchemaIndexDB] Loaded {len(rows)} rows, emb shape={emb.shape}")
    return SchemaIndexDB(rows, emb)


@dataclass
class CellIndexRow:
    db_id: str
    table_idx: int
    table_name_norm: str
    table_name_db: str
    col_id: int
    col_name_norm: str
    col_name_db: str
    col_type: str
    value: str
    freq: int
    text: str

class CellIndexDB:
    def __init__(self, rows: List[CellIndexRow], emb: np.ndarray):
        self.rows = rows
        self.emb = emb  # shape [N, dim]

def load_cell_index_db(
    jsonl_path: str = CELL_INDEX_JSONL,
    emb_path: str   = CELL_INDEX_EMB,
) -> CellIndexDB:
    rows: List[CellIndexRow] = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            rows.append(
                CellIndexRow(
                    db_id=obj["db_id"],
                    table_idx=int(obj["table_idx"]),
                    table_name_norm=obj["table_name_norm"],
                    table_name_db=obj["table_name_db"],
                    col_id=int(obj["col_id"]),
                    col_name_norm=obj["col_name_norm"],
                    col_name_db=obj["col_name_db"],
                    col_type=obj["col_type"],
                    value=str(obj["value"]),
                    freq=int(obj["freq"]),
                    text=obj["text"],
                )
            )
    emb = np.load(emb_path)
    assert len(rows) == emb.shape[0], f"cell_index: rows({len(rows)}) != emb({emb.shape[0]})"
    print(f"[CellIndexDB] Loaded {len(rows)} rows, emb shape={emb.shape}")
    return CellIndexDB(rows, emb)


# 실제 로딩
table_index_db  = load_table_index_db()
schema_index_db = load_schema_index_db()
cell_index_db   = load_cell_index_db()


[TableIndexDB] Loaded 876 rows, emb shape=(876, 1024)
[SchemaIndexDB] Loaded 4503 rows, emb shape=(4503, 1024)
[CellIndexDB] Loaded 1121733 rows, emb shape=(1121733, 1024)


In [ ]:
import re
from dataclasses import dataclass

# ====== Cell 4: 1단계 - QuestionAnalyzer & Token Budget (Spider Optimized) ======

@dataclass
class QuestionFeatures:
    q_raw: str
    q_norm: str
    is_agg: bool = False
    needs_join: bool = False
    is_long: bool = False
    has_group_by: bool = False

    # Spider 데이터셋 특화 필드 (난이도 'Hard', 'Extra Hard' 대응)
    needs_nesting: bool = False      # 서브쿼리/중첩 쿼리 필요 여부
    has_set_op: bool = False         # 집합 연산 (INTERSECT, UNION, EXCEPT)
    has_complex_cond: bool = False   # 복합 조건 (LIKE, OR, BETWEEN)

    # 종합 난이도 점수 (0~10+)
    difficulty_score: int = 0


class QuestionAnalyzer:
    """
    Spider 데이터셋(Cross-domain, Complex SQL)에 최적화된 질문 분석기.
    단순 키워드를 넘어 SQL의 구조적 난이도(Easy, Medium, Hard, Extra Hard)를
    추정하여 토큰 예산 분배의 근거를 마련합니다.
    """
    def analyze(self, q_raw: str) -> QuestionFeatures:
        q = q_raw.strip()
        q_low = q.lower()

        # 1. 기본 키워드 정의
        agg_keywords = ["count", "sum", "average", "avg", "max", "min", "total", "number of"]
        group_keywords = ["group by", "per ", "for each", "by department", "by type", "each"]

        # 2. 구조적 복잡도 키워드 (Spider Hard/Extra Hard 패턴)
        nesting_keywords = [
            "who have", "that have", "which has", "where the",
            "more than", "less than", "higher than", "lower than",
            "most", "least", "best", "worst", "top ", "bottom ",
            "not in", "except",
        ]

        # Set Operations (집합 연산)
        set_op_keywords = ["intersect", "union", "except", "both", "neither", "either", "common"]

        # Complex Conditions (복합 조건)
        complex_cond_keywords = [" or ", " and ", "between", "not in", "like", "start with", "end with"]

        # Spider 특화: 다중 테이블 참조 패턴 (조인 강도 측정)
        multi_entity_indicators = [
            " and ", " with ", " of ", " in ", " for ",
            "whose", "their", "its",
            "corresponding", "related", "associated"
        ]

        # --- Feature Extraction ---
        is_agg = any(k in q_low for k in agg_keywords)
        has_group_by = any(k in q_low for k in group_keywords)
        needs_nesting = any(k in q_low for k in nesting_keywords)
        has_set_op = any(k in q_low for k in set_op_keywords)
        has_complex_cond = any(k in q_low for k in complex_cond_keywords)

        # 조인 강도 측정
        is_long = len(q.split()) > 15
        multi_entity_score = sum(1 for k in multi_entity_indicators if k in q_low)
        needs_join = is_long or multi_entity_score >= 2

        # Spider 난이도 스코어 (0~12 범위)
        score = 0
        if is_agg: score += 1
        if needs_join:
            score += 2 + min(2, multi_entity_score)
        if has_group_by: score += 3
        if has_complex_cond: score += 2
        if needs_nesting: score += 4
        if has_set_op: score += 5

        if needs_nesting and (has_set_op or multi_entity_score >= 3):
            score += 2  # 중첩 + 다중 테이블 = 극복잡

        return QuestionFeatures(
            q_raw=q,
            q_norm=q_low,
            is_agg=is_agg,
            needs_join=needs_join,
            is_long=is_long,
            has_group_by=has_group_by,
            needs_nesting=needs_nesting,
            has_set_op=has_set_op,
            has_complex_cond=has_complex_cond,
            difficulty_score=score
        )

question_analyzer = QuestionAnalyzer()


def token_budget_controller(
    feats: QuestionFeatures,
    T_max: int = 4096,
):
    """
    Spider 데이터셋 통계 기반 토큰 분배 (2-way: schema / cell):
    - 스키마 요약(schemas)과 셀 요약(cells)에만 예산을 분배.
    - B_ctx 는 항상 0으로 고정 (현재 파이프라인에서는 사용하지 않음).
    """

    difficulty = feats.difficulty_score

    # --- 1) 난이도별 기본 분배 (schema / cell) ---
    if difficulty >= 10:  # Extra Hard
        # 구조가 훨씬 중요
        w_schema = 0.65
        w_cell   = 0.35
        # print(">> [Budget] Extra Hard Detected (Score 10+): Schema-focused")
    elif difficulty >= 7:  # Hard
        w_schema = 0.55
        w_cell   = 0.45
        # print(">> [Budget] Hard Detected (Score 7-9): Slightly schema-heavy")
    elif difficulty >= 4:  # Medium
        w_schema = 0.50
        w_cell   = 0.50
        # print(">> [Budget] Medium Detected (Score 4-6): Balanced")
    else:  # Easy
        w_schema = 0.35
        w_cell   = 0.65
        # print(">> [Budget] Easy Detected (Score < 4): Cell-focused")

    # --- 2) 조인 복잡도 반영 ---
    if feats.needs_join:
        join_boost = 0.10
        w_schema += join_boost
        w_cell   -= join_boost
        # print(f">> [Budget] Join detected: +{join_boost:.2f} to schema (from cell)")

    # --- 3) 집합 연산 / 중첩 쿼리 반영 ---
    if feats.has_set_op:
        # 집합 연산이 있을 땐 구조를 더 많이 보여줘야 함
        w_schema = max(w_schema, 0.70)
        # print(">> [Budget] Set operation: Schema >= 70% enforced")

    if feats.needs_nesting:
        w_schema += 0.10
        w_cell   -= 0.10
        # print(">> [Budget] Nesting detected: Schema prioritized")

    # --- 4) 안전한 범위로 클램핑 후 정규화 ---
    w_schema = max(0.20, min(0.80, w_schema))
    w_cell   = max(0.20, min(0.80, w_cell))

    total_w = w_schema + w_cell
    w_schema /= total_w
    w_cell   /= total_w

    # --- 5) 최종 토큰 계산 ---
    B_schema = int(T_max * w_schema)
    B_cell   = T_max - B_schema
    B_ctx    = 0  # 현재 파이프라인에서는 사용하지 않음

    # 디버깅용 출력 (원하면 주석 해제)
    # print(f"[BudgetController] Difficulty Score={feats.difficulty_score}")
    # print(f"[BudgetController] T_max={T_max} -> "
    #       f"B_schema={B_schema} ({w_schema:.2%}), "
    #       f"B_cell={B_cell} ({w_cell:.2%}), "
    #       f"B_ctx={B_ctx}")

    return B_schema, B_cell, B_ctx


In [ ]:
# ====== Cell 5: 2단계 - TableRetriever (DB 지정 가능 + 테이블 ranking) ======
import re
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional, Set
from collections import defaultdict

# 간단 토크나이저 (lexical schema linking용)
def simple_tokenize(text: str) -> List[str]:
    text = text.lower()
    text = re.sub(r"[^a-z0-9_]+", " ", text)
    return [t for t in text.split() if t]


@dataclass
class RetrievedTable:
    db_id: str
    table_name_db: str
    score: float              # 최종 re-ranking 스코어
    table_idx: int            # table_index_db에서의 인덱스
    sim_table: float          # 테이블 summary 임베딩 기반 유사도
    sim_col_max: float        # 질문 vs 컬럼 임베딩 최대 유사도
    lex_score: float          # lexical schema linking 스코어 (정규화된 값)


class TableRetriever:
    """
    테이블 검색기 (두 모드 지원):

    1) given_db_id=None  인 경우:
       - 전체 DB에 대해 간단한 DB-first 선택 + 그 DB 안에서 테이블 ranking
       - (기존 파이프라인이 사용하던 모드와 최대한 비슷하게 유지)

    2) given_db_id가 주어진 경우:
       - 그 DB 안의 테이블들만 대상으로 table / column / lexical 스코어를 계산해
         top_m 테이블을 반환.
       - Spider 기본 세팅처럼 "이미 DB id를 알고 시작"하는 상황에서 사용.
    """

    def __init__(
        self,
        table_db: TableIndexDB,
        schema_db: SchemaIndexDB,
        # 테이블/컬럼 레벨 re-ranking 가중치
        w_table: float = 0.5,
        w_col: float = 0.35,
        w_lex: float = 0.15,
        # DB 레벨 스코어 가중치 (given_db_id=None일 때만 사용)
        w_db_table: float = 0.4,
        w_db_col: float = 0.4,
        w_db_lex: float = 0.2,
    ):
        self.table_db = table_db
        self.schema_db = schema_db

        self.w_table = w_table
        self.w_col = w_col
        self.w_lex = w_lex

        self.w_db_table = w_db_table
        self.w_db_col = w_db_col
        self.w_db_lex = w_db_lex

        # (db_id) -> table_index에서의 인덱스 리스트
        self.db_to_table_indices: Dict[str, List[int]] = defaultdict(list)
        for idx, row in enumerate(self.table_db.rows):
            self.db_to_table_indices[row.db_id].append(idx)

        # (db_id, table_name_db) -> schema_index에서의 row 인덱스 리스트
        self.table_to_schema_indices: Dict[Tuple[str, str], List[int]] = defaultdict(list)
        # (db_id) -> schema_index에서의 row 인덱스 리스트
        self.db_to_schema_indices: Dict[str, List[int]] = defaultdict(list)
        for idx, row in enumerate(self.schema_db.rows):
            key = (row.db_id, row.table_name_db)
            self.table_to_schema_indices[key].append(idx)
            self.db_to_schema_indices[row.db_id].append(idx)

    # ---------- DB 레벨 lexical score ----------
    def _compute_db_lexical_score(
        self,
        q_tokens: List[str],
        db_id: str,
    ) -> float:
        """
        질문 토큰 vs 해당 DB 내의 (모든 테이블명 + 모든 컬럼명) 토큰 overlap 기반 lexical 스코어.
        - 테이블명 일치: 가중치 2
        - 컬럼명 일치: 가중치 1
        마지막에 질문 토큰 수로 나눠 정규화.
        """
        if not q_tokens:
            return 0.0

        table_tokens_set = set()
        col_tokens_set = set()

        # (1) 테이블명 토큰
        for t_idx in self.db_to_table_indices.get(db_id, []):
            t_row = self.table_db.rows[t_idx]
            table_tokens_set.update(simple_tokenize(t_row.table_name_norm))

        # (2) 컬럼명 토큰
        for s_idx in self.db_to_schema_indices.get(db_id, []):
            s_row = self.schema_db.rows[s_idx]
            col_tokens_set.update(simple_tokenize(s_row.col_name_norm))

        score = 0.0
        for w in q_tokens:
            if w in table_tokens_set:
                score += 2.0
            if w in col_tokens_set:
                score += 1.0

        return score / len(q_tokens)

    # ---------- 테이블 레벨 컬럼 임베딩 기반 최대 sim ----------
    def _compute_table_max_column_sim(
        self,
        schema_sims_all: np.ndarray,
        db_id: str,
        table_name_db: str,
    ) -> float:
        """
        schema_sims_all: schema_db.emb @ q_emb 결과 (shape: [N_schema])
        해당 (db_id, table_name_db) 테이블의 컬럼들 중
        질문 임베딩과 cosine similarity가 가장 큰 값을 반환.
        컬럼이 없으면 0.0.
        """
        key = (db_id, table_name_db)
        idx_list = self.table_to_schema_indices.get(key)
        if not idx_list:
            return 0.0

        sims = schema_sims_all[idx_list]  # (num_cols,)
        return float(np.max(sims))

    # ---------- 테이블 레벨 lexical score ----------
    def _compute_table_lexical_score(
        self,
        q_tokens: List[str],
        table_name_norm: str,
        columns: List[str],
    ) -> float:
        """
        질문 토큰 vs 해당 테이블의 (테이블명 + 컬럼명) 토큰 overlap 기반 lexical 스코어.
        - 테이블명 일치: 2점
        - 컬럼명 일치: 1점
        질문 토큰 수로 나눠 정규화.
        """
        if not q_tokens:
            return 0.0

        table_tokens = set(simple_tokenize(table_name_norm))
        col_tokens = set()
        for col in columns:
            col_tokens.update(simple_tokenize(col))

        score = 0.0
        for w in q_tokens:
            if w in table_tokens:
                score += 2.0
            if w in col_tokens:
                score += 1.0

        return score / len(q_tokens)

    # ---------- 주어진 DB 안에서 테이블 re-ranking ----------
    def _rank_tables_within_db(
        self,
        target_db: str,
        top_m: int,
        sim_table_all: np.ndarray,
        sim_schema_all: np.ndarray,
        q_tokens: List[str],
    ) -> List[RetrievedTable]:
        """
        target_db 안의 테이블들만 대상으로,
        sim_table + sim_col_max + lex_score를 섞어 최종 스코어를 계산하고 top_m 반환.
        """
        t_indices = self.db_to_table_indices.get(target_db, [])
        if not t_indices:
            return []

        scored_tables: List[Tuple[float, int, RetrievedTable]] = []

        for idx in t_indices:
            row = self.table_db.rows[idx]
            sim_table = float(sim_table_all[idx])

            # (a) 이 테이블의 컬럼들 중 질문과 가장 비슷한 column_sim
            sim_col_max = self._compute_table_max_column_sim(
                schema_sims_all=sim_schema_all,
                db_id=row.db_id,
                table_name_db=row.table_name_db,
            )

            # (b) 이 테이블에 대한 lexical score
            lex_score = self._compute_table_lexical_score(
                q_tokens=q_tokens,
                table_name_norm=row.table_name_norm,
                columns=row.columns,
            )

            # (c) 최종 테이블 스코어
            final_score = (
                self.w_table * sim_table +
                self.w_col   * sim_col_max +
                self.w_lex   * lex_score
            )

            rt = RetrievedTable(
                db_id=row.db_id,
                table_name_db=row.table_name_db,
                score=final_score,
                table_idx=idx,
                sim_table=sim_table,
                sim_col_max=sim_col_max,
                lex_score=lex_score,
            )
            scored_tables.append((final_score, idx, rt))

        scored_tables.sort(key=lambda x: -x[0])
        top = scored_tables[:top_m]
        results: List[RetrievedTable] = [tup[2] for tup in top]
        return results

    # ---------- 메인 retrieve 함수 ----------
    def retrieve(
        self,
        question: str,
        top_m: int = 10,
        given_db_id: Optional[str] = None,
    ) -> List[RetrievedTable]:
        """
        - given_db_id가 None이면:
            * 전체 DB에 대해 간단한 DB-level score를 계산해 best_db 선택 후
              그 DB 안에서만 테이블 re-ranking.
        - given_db_id가 주어지면:
            * 해당 DB 안에서만 테이블 re-ranking.
        """
        # 질문 임베딩
        q_emb = embed_texts([question])[0]              # (dim,)
        table_emb_all = self.table_db.emb               # (N_table, dim)
        schema_emb_all = self.schema_db.emb             # (N_schema, dim)

        # 모든 테이블/컬럼에 대해 질문과의 cosine similarity
        sim_table_all = table_emb_all @ q_emb           # (N_table,)
        sim_schema_all = schema_emb_all @ q_emb         # (N_schema,)

        # 질문 토큰 (lexical schema linking용)
        q_tokens = simple_tokenize(question)

        # ----- 모드 1: DB가 이미 주어진 경우 -----
        if given_db_id is not None:
            return self._rank_tables_within_db(
                target_db=given_db_id,
                top_m=top_m,
                sim_table_all=sim_table_all,
                sim_schema_all=sim_schema_all,
                q_tokens=q_tokens,
            )

        # ----- 모드 2: DB를 우리가 골라야 하는 경우 (간단 DB-first) -----
        db_scores: Dict[str, float] = {}
        for db_id, t_indices in self.db_to_table_indices.items():
            # (a) DB 내 테이블 중 질문과 가장 비슷한 table_sim
            if t_indices:
                db_table_max = float(np.max(sim_table_all[t_indices]))
            else:
                db_table_max = 0.0

            # (b) DB 내 컬럼 중 질문과 가장 비슷한 column_sim
            s_indices = self.db_to_schema_indices.get(db_id, [])
            if s_indices:
                db_col_max = float(np.max(sim_schema_all[s_indices]))
            else:
                db_col_max = 0.0

            # (c) DB 전체 lexical score
            db_lex = self._compute_db_lexical_score(q_tokens, db_id)

            # (d) 최종 DB 스코어
            db_score = (
                self.w_db_table * db_table_max +
                self.w_db_col   * db_col_max +
                self.w_db_lex   * db_lex
            )
            db_scores[db_id] = db_score

        if not db_scores:
            return []

        # 가장 점수가 높은 DB를 target_db로 선택
        target_db = max(db_scores.items(), key=lambda x: x[1])[0]

        # 그 DB 안에서 테이블 re-ranking
        return self._rank_tables_within_db(
            target_db=target_db,
            top_m=top_m,
            sim_table_all=sim_table_all,
            sim_schema_all=sim_schema_all,
            q_tokens=q_tokens,
        )

    @staticmethod
    def pick_db_id(retrieved: List[RetrievedTable]) -> Optional[str]:
        """
        retrieve 결과가 단일 DB에서만 나온다는 가정하에,
        첫 번째 결과의 db_id를 반환.
        (given_db_id를 쓴 경우에는 그대로 그 값이 나올 것)
        """
        if not retrieved:
            return None
        return retrieved[0].db_id


# 인스턴스 생성
table_retriever = TableRetriever(table_index_db, schema_index_db)

# 간단 테스트 (DB 미지정 모드)
q_test = "How many distinct publication dates are there in our record?"
feats_test = question_analyzer.analyze(q_test)
# retrieved_tables_test = table_retriever.retrieve(feats_test.q_norm, top_m=10)

gold_db_id =  "book_2"
retrieved_tables_test = table_retriever.retrieve(feats_test.q_norm, top_m=10)


print("\n[TableRetriever] Top-m tables:")
for t in retrieved_tables_test:
    print(
        f"- db={t.db_id:<25} table={t.table_name_db:<20} "
        f"score={t.score:.4f} | sim_table={t.sim_table:.4f} "
        f"| sim_col={t.sim_col_max:.4f} | lex={t.lex_score:.4f}"
    )

print("Predicted target_db:", TableRetriever.pick_db_id(retrieved_tables_test))



[TableRetriever] Top-m tables:
- db=book_2                    table=publication          score=0.4311 | sim_table=0.4480 | sim_col=0.4632 | lex=0.3000
- db=book_2                    table=book                 score=0.3302 | sim_table=0.3946 | sim_col=0.3796 | lex=0.0000
Predicted target_db: book_2


In [ ]:
# ====== Cell 6: SQL LLM 로드 및 토큰 트래커 ======
# ====== (새) LLM 로딩: Mistral-Nemo-Instruct-2407 & 토큰 트래커 ======
!pip install -q "transformers>=4.42.0" "accelerate>=0.31.0"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from contextlib import contextmanager

# Mistral-Nemo-Instruct-2407
SQL_MODEL_ID = "mistralai/Mistral-Nemo-Instruct-2407"

sql_tokenizer = AutoTokenizer.from_pretrained(SQL_MODEL_ID)
sql_model = AutoModelForCausalLM.from_pretrained(
    SQL_MODEL_ID,
    torch_dtype=torch.bfloat16,   # Colab Pro+ A100이면 bf16 충분
    device_map="auto",
)

# pad 토큰 없으면 eos와 동일하게 설정
if sql_tokenizer.pad_token_id is None:
    sql_tokenizer.pad_token = sql_tokenizer.eos_token

sql_model.eval()
print("Loaded SQL LLM:", SQL_MODEL_ID)


# --- DualTokenTracker: 모드별 input/output 토큰 수 추적 ---

class DualTokenTracker:
    def __init__(self):
        # mode -> {"input": int, "output": int, "total": int}
        self.data = {}

    def reset(self):
        self.data = {}

    @contextmanager
    def track(self, mode: str, input_ids: torch.Tensor):
        """
        input_ids: (1, L_in) 형태 텐서.
        컨텍스트 매니저가 끝날 때 holder["output_ids"]에서 out 토큰 수 계산.
        """
        in_tokens = int(input_ids.numel())
        holder = {}
        try:
            yield holder
        finally:
            output_ids = holder.get("output_ids", None)
            if output_ids is None:
                return
            total_tokens = int(output_ids.numel())
            out_tokens = max(0, total_tokens - in_tokens)

            rec = self.data.get(mode, {"input": 0, "output": 0, "total": 0})
            rec["input"] += in_tokens
            rec["output"] += out_tokens
            rec["total"] += in_tokens + out_tokens
            self.data[mode] = rec

    def print_summary(self):
        print("[TokenTracker] --- LLM token usage by mode ---")
        total_in = total_out = total = 0
        for mode, rec in self.data.items():
            print(
                f"  - {mode:<30} | "
                f"input={rec['input']:4d}, output={rec['output']:4d}, total={rec['total']:4d}"
            )
            total_in += rec["input"]
            total_out += rec["output"]
            total += rec["total"]
        print(
            f"  TOTAL{'':24} | "
            f"input={total_in:4d}, output={total_out:4d}, total={total:4d}"
        )


token_tracker = DualTokenTracker()


def build_sqlcoder_prompt(
    question: str,
    ddl_block: str,
    extra_instructions: str = "",
    extra_context: str = "",
) -> str:
    """
    Mistral Nemo에서 user 역할에 들어가는 content를 생성.
    (system 메시지는 sqlcoder_generate_from_prompt 안에서 공통으로 지정)
    """
    base_instr = (
        "You are an expert Text-to-SQL assistant. "
        "Given a natural language question and a database schema description, "
        "you must write a single SQL query that correctly answers the question."
    )
    if extra_instructions:
        base_instr += " " + extra_instructions

    extra_ctx = f"\n\nAdditional context:\n{extra_context}" if extra_context else ""

    user_content = f"""{base_instr}

    Question:
    {question}

    Database schema (summary):
    {ddl_block}{extra_ctx}

    Requirements:
    - Use only tables and columns that appear in the schema summary.
    - Do not invent new table or column names.
    - Return exactly one valid SQL query.
    - Wrap the query inside a ```sql code block, with nothing else.
    """
    return user_content


def sqlcoder_generate_from_prompt(
    mode: str,
    prompt: str,
    max_new_tokens: int = 256,
    temperature: float = 0.0,
) -> str:
    """
    Mistral-Nemo-Instruct-2407 + chat template 기반 생성 함수.
    - prompt: user 역할에 들어가는 content (질문 + 스키마 + 지시 포함).
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a precise assistant that writes SQL queries for the Spider Text-to-SQL task. "
                "Always follow the instructions and never output anything other than SQL when asked."
            ),
        },
        {"role": "user", "content": prompt},
    ]

    # HF chat template 사용 (model card 예시 기반) :contentReference[oaicite:1]{index=1}
    inputs = sql_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    )
    inputs = {k: v.to(sql_model.device) for k, v in inputs.items()}
    input_ids = inputs["input_ids"]

    # Nemo는 작은 temperature(0.3 이하)를 권장하지만,
    # SQL 안정성을 위해 기본값은 greedy(0.0)로 둔다.
    do_sample = temperature is not None and float(temperature) > 0.0
    temp = float(temperature) if temperature is not None else 0.0

    with torch.no_grad():
        with token_tracker.track(mode, input_ids) as holder:
            output_ids = sql_model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                temperature=temp,
                pad_token_id=sql_tokenizer.pad_token_id,
                eos_token_id=sql_tokenizer.eos_token_id,
            )
            holder["output_ids"] = output_ids

    # 입력 길이 이후로 생성된 토큰만 디코딩
    gen_ids = output_ids[0, input_ids.shape[1]:]
    text = sql_tokenizer.decode(gen_ids, skip_special_tokens=True)
    return text


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

The tokenizer you are loading from 'mistralai/Mistral-Nemo-Instruct-2407' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Loaded SQL LLM: mistralai/Mistral-Nemo-Instruct-2407


In [ ]:

# ====== Cell 7 (수정): 스키마/셀 Retriever + LLM 기반 Query Expansion ======
from dataclasses import dataclass
from typing import List, Set, Tuple, Dict
import json
import re

# ---------- 1. SelectedColumn / SelectedCell 정의 ----------

@dataclass
class SelectedColumn:
    db_id: str
    table_idx: int
    table_name_db: str
    col_id: int
    col_name_db: str
    col_type: str
    is_pk: bool
    is_fk: bool
    score: float


@dataclass
class SelectedCell:
    db_id: str
    table_idx: int
    table_name_db: str
    col_id: int
    col_name_db: str
    col_type: str
    value: str
    freq: int
    score: float


# ---------- 2. LLM 기반 Query Expander ----------

class QueryExpander:
    """
    TableRAG 스타일의 질의 확장기.
    - 입력: 원 질문 (자연어)
    - 출력: schema_query, cell_query 두 문자열
      * schema_query: 어떤 테이블/컬럼/개념이 중요한지 강조
      * cell_query  : 어떤 값/숫자/범주/조건이 중요한지 강조

    구현 상:
    - Mistral-Nemo(TextToSQL에 쓰이는 같은 LLM)를 사용해서
      간단한 JSON 응답을 받아온다.
    - 동일 질문에 대해 여러 번 호출되는 걸 막기 위해 내부 캐시 사용.
    """

    def __init__(self, max_new_tokens: int = 96):
        self.max_new_tokens = max_new_tokens
        self.cache: Dict[str, Tuple[str, str]] = {}

    def expand(self, question: str) -> Tuple[str, str]:
        """
        question -> (schema_query, cell_query)
        캐시가 있으면 재사용, 없으면 LLM 호출.
        """
        key = question.strip().lower()
        if key in self.cache:
            return self.cache[key]

        prompt = f"""You are helping a Text-to-SQL system (Spider dataset) to retrieve
relevant tables, columns, and cell values.

Given the following user question:

\"\"\"{question}\"\"\"


1. Write a SHORT schema-focused search query that highlights the main entities,
   table/column concepts, and relationships needed for this question.
2. Write a SHORT cell/value-focused search query that highlights important
   filter values, categories, numbers, or conditions.

Respond in **valid JSON** with the following format (no explanation):

{{
  "schema_query": "...",
  "cell_query": "..."
}}"""

        # sqlcoder_generate_from_prompt 는 이미 정의되어 있다고 가정
        raw = sqlcoder_generate_from_prompt(
            mode="pipeline_qexp",   # 토큰 트래킹용 모드 이름
            prompt=prompt,
            max_new_tokens=self.max_new_tokens,
        )

        # JSON 부분만 추출
        schema_q = ""
        cell_q = ""

        try:
            # 첫 번째 { ... } 블록 추출
            m = re.search(r"\{.*\}", raw, flags=re.DOTALL)
            if m:
                obj = json.loads(m.group(0))
                schema_q = str(obj.get("schema_query", "")).strip()
                cell_q = str(obj.get("cell_query", "")).strip()
        except Exception:
            # 실패하면 아주 단순한 폴백: 줄 단위로 첫 두 개 문장을 나눠 쓰기
            lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]
            if lines:
                schema_q = lines[0]
            if len(lines) > 1:
                cell_q = lines[1]

        # 안전장치: 비어 있으면 원 질문으로 대체
        if not schema_q:
            schema_q = question.strip()
        if not cell_q:
            cell_q = question.strip()

        self.cache[key] = (schema_q, cell_q)
        return schema_q, cell_q


# 전역 QueryExpander 인스턴스
query_expander = QueryExpander(max_new_tokens=96)


# ---------- 3. SchemaRetriever / CellRetriever ----------

class SchemaRetriever:
    def __init__(self, schema_db: SchemaIndexDB):
        self.db = schema_db

    def retrieve_for_db_tables(
        self,
        question: str,
        target_db_id: str,
        allowed_tables: Set[str],
        top_k: int = 50,
    ) -> List[SelectedColumn]:
        """
        - LLM 기반 query expansion으로 schema_query를 만들고,
          question + schema_query를 합친 질의 임베딩으로 컬럼 검색.
        - 대상: target_db_id + allowed_tables 에 속하는 컬럼들만.
        """
        # 1) 질의 확장
        schema_q, _ = query_expander.expand(question)
        # 2) 원 질문 + schema_query를 붙여서 임베딩
        q_for_schema = question.strip() + " ; " + schema_q
        q_emb = embed_texts([q_for_schema])[0]

        rows = self.db.rows
        emb = self.db.emb

        # 대상 db + 테이블만 필터링
        idx_list = [
            i for i, r in enumerate(rows)
            if r.db_id == target_db_id and r.table_name_db in allowed_tables
        ]
        if not idx_list:
            return []

        sub_emb = emb[idx_list]  # (M, dim)
        sims = sub_emb @ q_emb    # cosine similarity (정규화 임베딩 가정)
        order = np.argsort(-sims)[:top_k]

        selected: List[SelectedColumn] = []
        for k in order:
            idx = idx_list[k]
            r = rows[idx]
            score = float(sims[k])
            selected.append(
                SelectedColumn(
                    db_id=r.db_id,
                    table_idx=r.table_idx,
                    table_name_db=r.table_name_db,
                    col_id=r.col_id,
                    col_name_db=r.col_name_db,
                    col_type=r.col_type,
                    is_pk=r.is_pk,
                    is_fk=r.is_fk,
                    score=score,
                )
            )
        return selected


class CellRetriever:
    def __init__(self, cell_db: CellIndexDB):
        self.db = cell_db

    def retrieve_for_db_tables(
        self,
        question: str,
        target_db_id: str,
        allowed_tables: Set[str],
        top_k: int = 50,
    ) -> List[SelectedCell]:
        """
        - LLM 기반 query expansion으로 cell_query를 만들고,
          question + cell_query를 합친 질의 임베딩으로 셀 검색.
        - 대상: target_db_id + allowed_tables 에 속하는 셀들만.
        """
        # 1) 질의 확장
        _, cell_q = query_expander.expand(question)
        # 2) 원 질문 + cell_query를 붙여서 임베딩
        q_for_cell = question.strip() + " ; " + cell_q
        q_emb = embed_texts([q_for_cell])[0]

        rows = self.db.rows
        emb = self.db.emb

        idx_list = [
            i for i, r in enumerate(rows)
            if r.db_id == target_db_id and r.table_name_db in allowed_tables
        ]
        if not idx_list:
            return []

        sub_emb = emb[idx_list]
        sims = sub_emb @ q_emb
        order = np.argsort(-sims)[:top_k]

        selected: List[SelectedCell] = []
        for k in order:
            idx = idx_list[k]
            r = rows[idx]
            score = float(sims[k])
            selected.append(
                SelectedCell(
                    db_id=r.db_id,
                    table_idx=r.table_idx,
                    table_name_db=r.table_name_db,
                    col_id=r.col_id,
                    col_name_db=r.col_name_db,
                    col_type=r.col_type,
                    value=r.value,
                    freq=r.freq,
                    score=score,
                )
            )
        return selected


# 인스턴스 생성
schema_retriever = SchemaRetriever(schema_index_db)
cell_retriever   = CellRetriever(cell_index_db)



In [ ]:

# ====== Cell 8 (수정): 스키마 / 셀 컨텍스트 생성 (강압축 버전) ======
from collections import defaultdict
from typing import List, Tuple, Dict, Set


def build_schema_context_hybrid(
    schema_db: SchemaIndexDB,
    target_db_id: str,
    allowed_tables: Set[str],
    selected_columns: List[SelectedColumn],
    B_schema: int,
    tokenizer=sql_tokenizer,
) -> Tuple[str, int]:
    """
    파이프라인용 스키마 컨텍스트 생성기 (풀 DDL 사용 X).

    아이디어:
      - TableRAG-style query expansion + SchemaRetriever로 이미
        "질문과 가장 관련 높은 컬럼들"이 SelectedColumn 으로 들어옴.
      - 여기서:
          1) selected_columns 를 기준으로 '중요 테이블'만 우선 선택
          2) 각 테이블에서:
               - selected_columns
               - + 해당 테이블의 PK/FK 컬럼 (schema_db에서 조회)
             를 합쳐서, 짧은 한 줄짜리 요약:

             예: TABLE employee: emp_id[PK], dept_id[FK], name, age

      - B_schema 토큰 예산 안에서 table block을 최대한 많이 넣되,
        풀 DDL은 절대 사용하지 않음 (베이스라인과의 차별점).
    """
    if B_schema <= 0 or not allowed_tables:
        return "-- (no schema context)", 0

    # 1) selected_columns를 (db_id, table_name) 기준으로 그룹화
    table_to_sel: Dict[Tuple[str, str], List[SelectedColumn]] = defaultdict(list)
    for c in selected_columns:
        if c.db_id == target_db_id and c.table_name_db in allowed_tables:
            table_to_sel[(c.db_id, c.table_name_db)].append(c)

    # selected_columns 가 전혀 없을 수도 있으므로, allowed_tables 기반으로 최소 구성
    # (이 경우에는 PK/FK 위주로만 요약을 만든다)
    if not table_to_sel:
        for r in schema_db.rows:
            if (
                r.db_id == target_db_id
                and r.table_name_db in allowed_tables
                and r.col_id >= 0
                and (r.is_pk or r.is_fk)
            ):
                key = (r.db_id, r.table_name_db)
                # pseudo SelectedColumn (score=0)
                table_to_sel[key].append(
                    SelectedColumn(
                        db_id=r.db_id,
                        table_idx=r.table_idx,
                        table_name_db=r.table_name_db,
                        col_id=r.col_id,
                        col_name_db=r.col_name_db,
                        col_type=r.col_type,
                        is_pk=r.is_pk,
                        is_fk=r.is_fk,
                        score=0.0,
                    )
                )

    if not table_to_sel:
        return "-- (no schema rows for target_db)", 0

    # 2) 각 테이블 중요도(선택된 컬럼 수, 최대 score)로 정렬
    def table_importance(item):
        key, cols = item
        if not cols:
            return (0, 0.0)
        num_sel = len(cols)
        max_score = max(c.score for c in cols)
        return (num_sel, max_score)

    table_items = list(table_to_sel.items())
    table_items_sorted = sorted(
        table_items,
        key=lambda kv: table_importance(kv),
        reverse=True,
    )

    # 3) 테이블별로 PK/FK + selected_columns를 합쳐서 요약 문자열 생성
    blocks: List[str] = []
    used_tokens = 0

    # schema_db.rows 를 table_name 기준으로 빠르게 조회하기 위한 맵
    table_to_schema_rows: Dict[Tuple[str, str], List[SchemaIndexRow]] = defaultdict(list)
    for r in schema_db.rows:
        if r.db_id == target_db_id and r.table_name_db in allowed_tables and r.col_id >= 0:
            table_to_schema_rows[(r.db_id, r.table_name_db)].append(r)

    for (db_id, table_name), sel_cols in table_items_sorted:
        # 해당 테이블의 전체 스키마 메타
        schema_rows = table_to_schema_rows.get((db_id, table_name), [])

        # (1) sel_cols 기반 컬럼 정보
        col_map: Dict[str, Dict] = {}
        for c in sel_cols:
            col_map.setdefault(c.col_name_db, {
                "col_name": c.col_name_db,
                "is_pk": c.is_pk,
                "is_fk": c.is_fk,
                "score": c.score,
                "from_sel": True,
            })
            # 혹시 기존에 있던 값보다 score가 크면 갱신
            if c.score > col_map[c.col_name_db]["score"]:
                col_map[c.col_name_db]["score"] = c.score
            # PK/FK는 OR
            col_map[c.col_name_db]["is_pk"] = col_map[c.col_name_db]["is_pk"] or c.is_pk
            col_map[c.col_name_db]["is_fk"] = col_map[c.col_name_db]["is_fk"] or c.is_fk

        # (2) 해당 테이블의 PK/FK 컬럼을 추가 (selected_columns에 없더라도)
        for r in schema_rows:
            if not (r.is_pk or r.is_fk):
                continue
            if r.col_name_db not in col_map:
                col_map[r.col_name_db] = {
                    "col_name": r.col_name_db,
                    "is_pk": r.is_pk,
                    "is_fk": r.is_fk,
                    "score": 0.0,
                    "from_sel": False,
                }
            else:
                col_map[r.col_name_db]["is_pk"] = col_map[r.col_name_db]["is_pk"] or r.is_pk
                col_map[r.col_name_db]["is_fk"] = col_map[r.col_name_db]["is_fk"] or r.is_fk

        if not col_map:
            # 이 테이블에서는 쓸 컬럼이 없음
            continue

        cols_info = list(col_map.values())

        # (3) 컬럼 정렬: PK > FK > (질문에서 선택됨) > score > 이름
        def col_rank(ci):
            return (
                0 if ci["is_pk"] else 1,
                0 if ci["is_fk"] else 1,
                0 if ci["from_sel"] else 1,
                -ci["score"],
                ci["col_name"].lower(),
            )

        cols_info_sorted = sorted(cols_info, key=col_rank)

        # (4) 한 줄짜리 테이블 요약 만들기
        col_tokens = []
        for ci in cols_info_sorted:
            tags = []
            if ci["is_pk"]:
                tags.append("PK")
            if ci["is_fk"]:
                tags.append("FK")
            if tags:
                col_tokens.append(f"{ci['col_name']}[{','.join(tags)}]")
            else:
                col_tokens.append(ci["col_name"])

        block = f"TABLE {table_name}: " + ", ".join(col_tokens)

        # (5) 토큰 예산 체크
        n_tok = len(tokenizer(block, return_tensors="pt").input_ids[0])
        if used_tokens + n_tok > B_schema:
            # 이 테이블 블록을 추가하면 예산을 초과하므로 중단
            break

        blocks.append(block)
        used_tokens += n_tok

    if not blocks:
        return "-- (schema budget too small)", 0

    schema_context = "\n".join(blocks)
    return schema_context, used_tokens


def build_cell_context_from_selected_cells(
    selected_cells: List[SelectedCell],
    B_cell: int,
    tokenizer=sql_tokenizer,
    max_values_per_column: int = 4,
) -> Tuple[str, int]:
    """
    SelectedCell -> '압축된 셀 요약' 문자열.

    설계:
      - 컬럼 단위로 그룹화
      - 각 컬럼에 대해 score가 높은 셀 값 몇 개만 선택
      - 형식 예:
          COL employee.status: ['ACTIVE', 'INACTIVE']
          COL competition.theme: ['Aliens', 'Robots']
      - B_cell 토큰 예산 안에서 column-block을 가능한 한 많이 채우기.
    """
    if not selected_cells or B_cell <= 0:
        return "", 0

    grouped: Dict[Tuple[str, str], List[SelectedCell]] = defaultdict(list)
    for cell in selected_cells:
        grouped[(cell.table_name_db, cell.col_name_db)].append(cell)

    blocks: List[str] = []
    used_tokens = 0

    # 컬럼 중요도: 그 컬럼에서 가장 높은 score 를 기준으로 정렬
    col_keys_sorted = sorted(
        grouped.keys(),
        key=lambda key: -max(c.score for c in grouped[key]),
    )

    for (table_name, col_name) in col_keys_sorted:
        cells = grouped[(table_name, col_name)]
        cells_sorted = sorted(cells, key=lambda c: -c.score)

        # unique 값 모으기
        values = []
        seen = set()
        for c in cells_sorted:
            v = str(c.value)
            if v in seen:
                continue
            seen.add(v)
            if len(v) > 40:
                v = v[:37] + "..."
            values.append(v)
            if len(values) >= max_values_per_column:
                break

        if not values:
            continue

        # block 형식: "COL table.col: ['v1', 'v2', ...]"
        val_strs = [repr(v) for v in values]
        block = f"COL {table_name}.{col_name}: [{', '.join(val_strs)}]"
        n_tok = len(tokenizer(block, return_tensors="pt").input_ids[0])

        if used_tokens + n_tok > B_cell:
            # 더 넣으면 예산 초과 → 종료
            break

        blocks.append(block)
        used_tokens += n_tok

        if used_tokens >= B_cell:
            break

    if not blocks:
        return "", used_tokens

    cell_context = "\n".join(blocks)
    return cell_context, used_tokens



In [ ]:
# ====== (새) TextToSQLAgent: Mistral Nemo 기반 Text-to-SQL (공용 경량 self-check) ======
import re
from dataclasses import dataclass

def extract_sql_from_output(text: str) -> str:
    """
    LLM 출력에서 ```sql ... ``` 블록을 우선적으로 추출.
    없으면 첫 번째 ``` 이전까지, 그것도 없으면 전체를 반환.
    """
    code_blocks = re.findall(
        r"```sql\s*(.*?)```", text, flags=re.IGNORECASE | re.DOTALL
    )
    if code_blocks:
        return code_blocks[0].strip()
    if "```" in text:
        return text.split("```")[0].strip()
    return text.strip()


@dataclass
class SQLResult:
    sql: str            # 최종 SQL (self-correction 후)
    sql_initial: str    # 1차 생성 SQL
    ddl_and_cells: str  # main 패스에서 사용한 전체 컨텍스트 (DDL + cell)
    ddl_only: str       # self-correction에서 사용한 스키마 요약만


class TextToSQLAgent:
    """
    Mistral-Nemo-Instruct-2407 기반 Text-to-SQL 모듈.

    - main 패스:
        압축/풀 스키마 요약(ddl_block) + (필요 시) 셀 요약(cell_context)을 모두 사용해 초안 SQL 생성.
    - self-correction (베이스라인/파이프라인 공통):
        셀 요약 없이, 스키마 요약(ddl_only) + 질문 + draft SQL만 사용해
        존재하지 않는 컬럼/테이블/조건 오류 등을 가볍게 수정.
    """

    def __init__(self, mode_prefix: str, extra_instructions: str = ""):
        """
        mode_prefix: 토큰 트래커에 찍힐 prefix (e.g. 'baseline', 'pipeline')
        extra_instructions: SQL 생성시 추가로 붙일 지시문 (옵션)
        """
        self.mode_prefix = mode_prefix
        self.extra_instructions = extra_instructions

    # ---- 1차 SQL 생성 ----
    def _generate_main_sql(
        self,
        question: str,
        ddl_and_cells: str,
        max_new_tokens: int = 256,
    ) -> str:
        """
        ddl_and_cells = ddl_block (+ 선택적 cell_context)를 모두 포함한 컨텍스트.
        build_sqlcoder_prompt / sqlcoder_generate_from_prompt 는
        미리 정의된 헬퍼를 그대로 사용한다고 가정.
        """
        user_prompt = build_sqlcoder_prompt(
            question=question,
            ddl_block=ddl_and_cells,
            extra_instructions=self.extra_instructions,
            extra_context="",
        )
        mode = f"{self.mode_prefix}_sql_main"
        out_text = sqlcoder_generate_from_prompt(
            mode=mode,
            prompt=user_prompt,
            max_new_tokens=max_new_tokens,
        )
        sql = extract_sql_from_output(out_text)
        return sql

    # ---- self-correction (경량, 베이스라인/파이프라인 공통) ----
    def _self_correct_sql(
        self,
        question: str,
        ddl_only: str,
        draft_sql: str,
        max_new_tokens: int = 256,
    ) -> str:
        """
        공용 경량 self-correction:
        - 스키마 요약(ddl_only) + 질문 + draft SQL만 사용.
        - 셀 컨텍스트는 사용하지 않음.
        """
        correction_prompt = f"""We have the following database schema summary:

        {ddl_only}

        The user asked the following question:
        `{question}`

        The following SQL query was generated, but it may contain mistakes
        (e.g., using columns that do not exist in the schema, or using incorrect numeric conditions):

        ```sql
        {draft_sql}
        Please fix this SQL so that:

        It uses only tables and columns that appear in the schema summary above.

        All numeric conditions are consistent with the user's question.

        The query is syntactically valid SQL (SQLite/PostgreSQL compatible).

        If the original query is already correct, you may return it unchanged.

        Return ONLY the corrected SQL query, inside a single sql code block.
        """
        mode = f"{self.mode_prefix}_sql_selfcheck"
        out_text = sqlcoder_generate_from_prompt(
        mode=mode,
        prompt=correction_prompt,
        max_new_tokens=max_new_tokens,
        )
        sql = extract_sql_from_output(out_text)
        return sql


    # ---- 외부에서 쓰는 메인 엔트리 ----
    def generate_sql(
        self,
        question: str,
        ddl_block: str,
        cell_context: str,
        max_new_tokens: int = 256,
    ) -> SQLResult:
        """
        - ddl_block    : 스키마 요약 (베이스라인이면 풀 DDL, 파이프라인이면 압축 DDL 등)
        - cell_context : 압축된 셀 요약 (필요한 경우에만, 이미 예산 안에서 잘려 있음)

        main 패스는 ddl_block + (옵션) cell_context 모두 사용,
        self-correction은 ddl_block(=ddl_only)만 사용.
        """
        question = question.strip()

        if cell_context:
            ddl_and_cells = ddl_block + "\n\n-- Cell hints\n" + cell_context
        else:
            ddl_and_cells = ddl_block

        # 1차 초안 SQL
        draft_sql = self._generate_main_sql(
            question=question,
            ddl_and_cells=ddl_and_cells,
            max_new_tokens=max_new_tokens,
        )

        # 2차 self-correction (경량, ddl_only만 사용)
        final_sql = self._self_correct_sql(
            question=question,
            ddl_only=ddl_block,
            draft_sql=draft_sql,
            max_new_tokens=max_new_tokens,
        )

        return SQLResult(
            sql=final_sql,
            sql_initial=draft_sql,
            ddl_and_cells=ddl_and_cells,
            ddl_only=ddl_block,
        )

In [ ]:
# ====== Cell 9: 전체 파이프라인 (로그 버전 + 무로그 버전, db_id 지원, 디버깅 출력 강화) ======
import re
from typing import List, Set, Dict, Any, Optional

import textwrap  # 긴 프롬프트 pretty print 용


def should_use_cell_context(feats: QuestionFeatures, question: str) -> bool:
    """
    셀 컨텍스트를 쓸지 말지 결정하는 간단한 게이트.
    - 숫자 / 비교 연산 / 집계 여부 등을 기준으로 판단.
    """
    q = question.lower()

    # 숫자 등장 여부
    if re.search(r"\d", q):
        return True

    # 비교/조건 키워드
    cond_keywords = [
        "greater than", "less than", "more than", "at least", "at most", "no more than",
        "no less than", "before", "after", "older than", "newer than", "between",
        "equal to", "exactly", "minimum", "maximum",
        ">=", "<=", ">", "<", "=",
    ]
    if any(k in q for k in cond_keywords):
        return True

    # 집계 쿼리는 셀 정보가 도움이 될 확률이 더 높음
    if feats.is_agg:
        return True

    return False


def _run_full_pipeline_impl(
    question: str,
    T_max: int = 600,
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    verbose: bool = True,
    db_id: Optional[str] = None,   # ✅ gold dbid를 넘길 수 있음
) -> Dict[str, Any]:
    """
    공통 구현:
      - verbose=True  → run_full_pipeline (로그 출력)
      - verbose=False → run_full_pipeline_nologs (조용한 버전)

    흐름:
      0) 질문 단위로 token_tracker 리셋
      1) QuestionAnalyzer → B_schema, B_cell, B_ctx (예산)
      2) db_id에 따라 DB/테이블 검색
      3) SchemaRetriever / CellRetriever (LLM 기반 query expansion 사용)
      4) build_schema_context_hybrid / build_cell_context_from_selected_cells 로
         압축 스키마/셀 컨텍스트 생성
      5) TextToSQLAgent(mode_prefix="pipeline")로 main + 경량 self-check
      6) token_tracker 에서 토큰 사용량 수집
    """
    # ---------------- 0) 토큰 트래커 리셋 (질문당 1번) ----------------
    token_tracker.reset()

    # ---------------- 1) 질문 분석 + Budget ----------------
    feats = question_analyzer.analyze(question)
    B_schema, B_cell, B_ctx = token_budget_controller(feats, T_max=T_max)

    if verbose:
        print("=" * 70)
        print("[Pipeline] Question")
        print("- Raw :", repr(question))
        print("- Norm:", repr(feats.q_norm))
        print()
        print("[1] Question features")
        print(f"  - is_agg       : {feats.is_agg}")
        print(f"  - needs_join   : {feats.needs_join}")
        print(f"  - is_long      : {feats.is_long}")
        print(f"  - has_group_by : {feats.has_group_by}")
        print(f"  - needs_nesting: {feats.needs_nesting}")
        print()
        print(f"[1] Token budgets from controller (T_max={T_max})")
        print(f"  - B_schema : {B_schema}")
        print(f"  - B_cell   : {B_cell}")
        print(f"  - B_ctx    : {B_ctx}  (intended total for schema+cell)")
        print(f"  - B_schema + B_cell : {B_schema + B_cell}")

    # ---------------- 2) 테이블 검색 ----------------
    if db_id is None:
        # ✅ 기존 로직: DB-first TableRetriever 사용
        retrieved_tables = table_retriever.retrieve(feats.q_norm, top_m=top_m_tables)
        target_db = TableRetriever.pick_db_id(retrieved_tables)
        if target_db is None:
            raise RuntimeError("No target_db selected in pipeline.")
        allowed_tables: Set[str] = {
            t.table_name_db for t in retrieved_tables if t.db_id == target_db
        }
    else:
        # ✅ 새 로직: gold db_id가 주어졌으므로 해당 DB 안에서만 테이블 검색
        target_db = db_id

        # question → 임베딩
        q_emb = embed_texts([feats.q_norm])[0]   # (dim,)

        rows = table_index_db.rows
        emb  = table_index_db.emb

        # 주어진 db_id에 속하는 테이블들만 후보
        idx_list = [i for i, r in enumerate(rows) if r.db_id == target_db]
        if not idx_list:
            raise RuntimeError(f"No tables found in table_index_db for db_id={target_db}")

        sub_emb = emb[idx_list]
        sims = sub_emb @ q_emb
        order = np.argsort(-sims)[:top_m_tables]

        retrieved_tables = []
        for k in order:
            idx = idx_list[k]
            r = rows[idx]
            sim_table = float(sims[k])

            # RetrievedTable 시그니처가 (db_id, table_name_db, score, table_idx, sim_table, sim_col_max, lex_score)
            # 인 경우와 (db_id, table_name_db, score) 인 경우 모두 대응
            try:
                rt = RetrievedTable(
                    db_id=r.db_id,
                    table_name_db=r.table_name_db,
                    score=sim_table,
                    table_idx=idx,
                    sim_table=sim_table,
                    sim_col_max=0.0,
                    lex_score=0.0,
                )
            except TypeError:
                # 예전 3-필드 버전인 경우
                rt = RetrievedTable(
                    db_id=r.db_id,
                    table_name_db=r.table_name_db,
                    score=sim_table,
                )
            retrieved_tables.append(rt)

        allowed_tables = {t.table_name_db for t in retrieved_tables}

    if verbose:
        print("\n[2] Table retrieval")
        print(f"  - db_id given? : {db_id is not None}")
        print(f"  - target_db    : {target_db}")
        print(f"  - top_m_tables : {top_m_tables}")
        print("  - Retrieved tables:")
        for i, t in enumerate(retrieved_tables):
            print(f"    [{i:02d}] db={t.db_id:<20} table={t.table_name_db:<25} score={t.score:.4f}")
        print(f"  - allowed_tables ({len(allowed_tables)}): {sorted(allowed_tables)}")

    # ---------------- 3) 스키마 / 셀 Retrieval (LLM Q-exp 사용) ----------------
    selected_columns = schema_retriever.retrieve_for_db_tables(
        question=feats.q_raw,
        target_db_id=target_db,
        allowed_tables=allowed_tables,
        top_k=top_k_cols,
    )

    use_cell = should_use_cell_context(feats, question)
    B_cell_eff = B_cell if use_cell else 0

    selected_cells: List[SelectedCell] = []
    if B_cell_eff > 0:
        selected_cells = cell_retriever.retrieve_for_db_tables(
            question=feats.q_raw,
            target_db_id=target_db,
            allowed_tables=allowed_tables,
            top_k=top_k_cells,
        )

    if verbose:
        print("\n[3] Schema / Cell retrieval (after LLM query expansion)")
        print(f"  - top_k_cols        : {top_k_cols}")
        print(f"  - #selected_columns : {len(selected_columns)}")
        for i, col in enumerate(selected_columns[:10]):
            print(f"    [col {i:02d}] {col}")
        if len(selected_columns) > 10:
            print(f"    ... (+{len(selected_columns) - 10} more columns)")

        print()
        print(f"  - should_use_cell_context : {use_cell}")
        print(f"  - B_cell_eff (actual B_cell used) : {B_cell_eff}")
        print(f"  - top_k_cells         : {top_k_cells}")
        print(f"  - #selected_cells     : {len(selected_cells)}")
        for i, cell in enumerate(selected_cells[:10]):
            print(f"    [cell {i:02d}] {cell}")
        if len(selected_cells) > 10:
            print(f"    ... (+{len(selected_cells) - 10} more cells)")

    # ---------------- 4) 스키마 / 셀 컨텍스트 생성 (압축) ----------------
    schema_context, schema_tokens_used = build_schema_context_hybrid(
        schema_db=schema_index_db,
        target_db_id=target_db,
        allowed_tables=allowed_tables,
        selected_columns=selected_columns,
        B_schema=B_schema,
        tokenizer=sql_tokenizer,
    )

    cell_context, cell_tokens_used = build_cell_context_from_selected_cells(
        selected_cells=selected_cells,
        B_cell=B_cell_eff,
        tokenizer=sql_tokenizer,
    )

    ctx_tokens_total = schema_tokens_used + cell_tokens_used

    if verbose:
        print("\n[4] Compressed schema / cell context")
        print(f"  - schema_tokens_used : {schema_tokens_used} / B_schema={B_schema}")
        print(f"  - cell_tokens_used   : {cell_tokens_used} / B_cell_eff={B_cell_eff}")
        print(f"  - total_ctx_tokens   : {ctx_tokens_total} (schema+cell)")
        print(f"  - B_ctx (controller) : {B_ctx}")

        # 질문/스키마/셀 각각에 대해 토큰 수 계산
        try:
            q_tokens = len(sql_tokenizer.encode(question))
            ddl_tokens = len(sql_tokenizer.encode(schema_context)) if schema_context else 0
            cell_tokens = len(sql_tokenizer.encode(cell_context)) if cell_context else 0
        except Exception:
            q_tokens = ddl_tokens = cell_tokens = -1

        print("\n[4-1] Final LLM input context token stats (approx. sql_tokenizer 기준)")
        print(f"  - question_tokens           : {q_tokens}")
        print(f"  - schema_ctx_tokens(re-enc) : {ddl_tokens}")
        print(f"  - cell_ctx_tokens  (re-enc) : {cell_tokens}")
        print(f"  - total_ctx_tokens_with_q   : {q_tokens + ddl_tokens + cell_tokens}")

        print("\n[Schema context preview]")
        print("-" * 40)
        if schema_context:
            print(schema_context[:800] + ("...\n" if len(schema_context) > 800 else "\n"))
        else:
            print("(empty)\n")

        print("[Cell context preview]")
        print("-" * 40)
        if cell_context:
            print(cell_context[:800] + ("...\n" if len(cell_context) > 800 else "\n"))
        else:
            print("(empty)\n")

    # ---------------- 5) Text-to-SQL (main + 경량 self-check) ----------------
    # ❌ 더 이상 여기서 reset 하지 않는다 (쿼리 확장 LLM 토큰까지 포함하기 위해)
    # token_tracker.reset()

    agent = TextToSQLAgent(
        mode_prefix="pipeline",
        extra_instructions=(
            "Use only the tables and columns from the schema summary. "
            "Do not invent new table or column names. "
            "Make sure numeric conditions match the question."
        ),
    )

    if verbose:
        print("\n[5] SQL generation (pipeline)")
        print("[5-0] LLM main call input (high-level)")
        print(f"  - question     : {repr(question.strip())}")
        print(f"  - ddl_block len: {len(schema_context)} chars")
        print(f"  - cell_ctx len : {len(cell_context)} chars")
        print("  - extra_instructions:")
        print("    " + agent.extra_instructions)

    sql_result = agent.generate_sql(
        question=question.strip(),
        ddl_block=schema_context,
        cell_context=cell_context,
        max_new_tokens=max_new_tokens_sql,
    )

    if verbose:
        print("\n[5-0-1] (Optional) Actual prompts if exposed by agent/sql_result")
        if hasattr(sql_result, "prompt_main"):
            print("\n[Prompt - main SQL generation]")
            print("-" * 50)
            print(textwrap.indent(sql_result.prompt_main, prefix="    "))
        elif hasattr(agent, "last_main_prompt"):
            print("\n[Prompt - main SQL generation (agent.last_main_prompt)]")
            print("-" * 50)
            print(textwrap.indent(agent.last_main_prompt, prefix="    "))
        else:
            print("  - main prompt not exposed; consider adding debug fields in TextToSQLAgent.")

        if hasattr(sql_result, "prompt_selfcheck"):
            print("\n[Prompt - self-check]")
            print("-" * 50)
            print(textwrap.indent(sql_result.prompt_selfcheck, prefix="    "))
        elif hasattr(agent, "last_selfcheck_prompt"):
            print("\n[Prompt - self-check (agent.last_selfcheck_prompt)]")
            print("-" * 50)
            print(textwrap.indent(agent.last_selfcheck_prompt, prefix="    "))
        else:
            print("  - self-check prompt not exposed; "
                  "add e.g. sql_result.prompt_selfcheck or agent.last_selfcheck_prompt for debugging.")

        print("\n[5-1] Initial SQL (draft):")
        print("-" * 40)
        print(sql_result.sql_initial)

        print("\n[5-2] Final SQL after self-correction:")
        print("-" * 40)
        print(sql_result.sql)

        print("\n[6] LLM token usage summary (pipeline)")
        print("-" * 40)
        token_tracker.print_summary()

        print("\n[6-1] token_tracker.data (raw)")
        print(token_tracker.data)

    # ---------------- 6) 결과 패키징 ----------------
    result = {
        "feats": feats,
        "retrieved_tables": retrieved_tables,
        "target_db": target_db,
        "allowed_tables": allowed_tables,
        "selected_columns": selected_columns,
        "selected_cells": selected_cells,
        "schema_context": schema_context,
        "cell_context": cell_context,
        "sql_result": sql_result,
        "token_usage": token_tracker.data,
        "budgets": {
            "B_schema": B_schema,
            "B_cell": B_cell,
            "B_ctx": B_ctx,
            "T_max": T_max,
        },
        "ctx_tokens_used": {
            "schema": schema_tokens_used,
            "cell": cell_tokens_used,
            "total": ctx_tokens_total,
        },
    }
    return result


def run_full_pipeline(
    question: str,
    T_max: int = 500,
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    db_id: Optional[str] = None,   # ✅ gold dbid 전달 가능
):
    """
    로그를 출력하는 버전.
    """
    return _run_full_pipeline_impl(
        question=question,
        T_max=T_max,
        top_m_tables=top_m_tables,
        top_k_cols=top_k_cols,
        top_k_cells=top_k_cells,
        max_new_tokens_sql=max_new_tokens_sql,
        verbose=True,
        db_id=db_id,
    )


def run_full_pipeline_nologs(
    question: str,
    T_max: int = 500,
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    db_id: Optional[str] = None,   # ✅ gold dbid 전달 가능
):
    """
    평가용/배치 실행용 조용한 버전.
    Spider 평가 코드에서 (sql, db_id) 튜플만 뽑을 때 이 함수를 래핑해서 사용.
    """
    return _run_full_pipeline_impl(
        question=question,
        T_max=T_max,
        top_m_tables=top_m_tables,
        top_k_cols=top_k_cols,
        top_k_cells=top_k_cells,
        max_new_tokens_sql=max_new_tokens_sql,
        verbose=False,
        db_id=db_id,
    )


In [ ]:
# @title
# ====== Cell 10: Baseline - Full Schema (db_id 지원 버전) ======
from collections import defaultdict
from typing import Optional

def build_full_schema_context_from_schema_index(
    schema_db: SchemaIndexDB,
    target_db_id: str,
    allowed_tables: Set[str],
    tokenizer=sql_tokenizer,
) -> Tuple[str, int]:
    """
    SchemaIndexDB에서 target_db_id + allowed_tables 에 해당하는
    '전체 스키마(모든 컬럼)'를 DDL 스타일 텍스트로 만든다.
    """
    grouped: Dict[str, List[SchemaIndexRow]] = defaultdict(list)

    for r in schema_db.rows:
        if r.db_id != target_db_id:
            continue
        if allowed_tables and r.table_name_db not in allowed_tables:
            continue
        grouped[r.table_name_db].append(r)

    lines = []
    for table_name, cols in grouped.items():
        lines.append(f"-- DB: {target_db_id}")
        lines.append(f"CREATE TABLE {table_name} (")

        col_lines = []
        for c in cols:
            col_type = (c.col_type or "TEXT").upper()
            ann = []
            if c.is_pk:
                ann.append("PK")
            if c.is_fk:
                ann.append("FK")
            comment = f" -- {'/'.join(ann)}" if ann else ""
            col_lines.append(f"  {c.col_name_db} {col_type}{comment}")

        for i, line in enumerate(col_lines):
            if i < len(col_lines) - 1:
                lines.append(line + ",")
            else:
                lines.append(line)

        lines.append(");")
        lines.append("")

    ddl_block = "\n".join(lines).strip()
    if ddl_block:
        n_tokens = len(tokenizer(ddl_block, return_tensors="pt").input_ids[0])
    else:
        n_tokens = 0

    return ddl_block, n_tokens

import textwrap  # 디버깅용 prompt pretty print

def run_baseline_full_schema(
    question: str,
    T_max: int = 2048,
    top_m_tables: int = 10,
    max_new_tokens_sql: int = 256,
    db_id: Optional[str] = None,   # ✅ gold dbid를 옵션으로 받음
):
    """
    베이스라인 파이프라인:
      - db_id가 None이면: 기존처럼 TableRetriever로 DB 선택
      - db_id가 주어지면: 해당 DB의 테이블들만 사용

    🔍 디버깅 모드: 파이프라인에서 일어나는 모든 단계와
       토큰/컨텍스트/프롬프트 등을 최대한 자세히 출력.
    """
    print("=" * 70)
    print("[Baseline] Question")
    print("  - Raw :", repr(question))
    # 1) 질문 분석 + 예산 (여기서는 참고용)
    feats = question_analyzer.analyze(question)
    print("  - Norm:", repr(feats.q_norm))

    B_schema, B_cell, B_ctx = token_budget_controller(feats, T_max=T_max)

    print("\n[Baseline 1] Question features & budgets")
    print(f"  - is_agg       : {feats.is_agg}")
    print(f"  - needs_join   : {feats.needs_join}")
    print(f"  - is_long      : {feats.is_long}")
    print(f"  - has_group_by : {feats.has_group_by}")
    print(f"  - needs_nesting: {feats.needs_nesting}")
    print()
    print(f"  - T_max        : {T_max}")
    print(f"  - B_schema     : {B_schema}")
    print(f"  - B_cell       : {B_cell}")
    print(f"  - B_ctx        : {B_ctx} (intended schema+cell)")
    print(f"  - B_schema + B_cell : {B_schema + B_cell}")

    # 2) 테이블 / DB 선택
    if db_id is None:
        # 기존 로직: TableRetriever를 사용해 DB-first
        retrieved_tables = table_retriever.retrieve(feats.q_norm, top_m=top_m_tables)

        print("\n[Baseline 2] Table retrieval (DB-first, TableRetriever)")
        print(f"  - top_m_tables : {top_m_tables}")
        print("  - Retrieved tables:")
        for i, t in enumerate(retrieved_tables):
            print(f"    [{i:02d}] db={t.db_id:<20} table={t.table_name_db:<25} score={t.score:.4f}")

        target_db = TableRetriever.pick_db_id(retrieved_tables)
        print(f"  => target_db : {target_db}")
        if target_db is None:
            raise RuntimeError("No target_db selected in baseline.")

        allowed_tables: Set[str] = {
            t.table_name_db for t in retrieved_tables if t.db_id == target_db
        }
    else:
        # db_id가 주어지면: 그 DB 안의 모든 테이블을 allowed_tables로 사용
        target_db = db_id

        # table_index_db에서 해당 DB의 모든 테이블명 수집
        table_names_for_db = sorted({
            row.table_name_db for row in table_index_db.rows
            if row.db_id == target_db
        })
        if not table_names_for_db:
            raise RuntimeError(f"[Baseline] No tables found for db_id={target_db} in table_index_db.")

        allowed_tables: Set[str] = set(table_names_for_db)

        # retrieved_tables 도 결과 dict에 넣을 수 있도록 대강 구성
        retrieved_tables = []
        for idx, row in enumerate(table_index_db.rows):
            if row.db_id != target_db:
                continue
            try:
                rt = RetrievedTable(
                    db_id=row.db_id,
                    table_name_db=row.table_name_db,
                    score=0.0,
                    table_idx=idx,
                    sim_table=0.0,
                    sim_col_max=0.0,
                    lex_score=0.0,
                )
            except TypeError:
                rt = RetrievedTable(
                    db_id=row.db_id,
                    table_name_db=row.table_name_db,
                    score=0.0,
                )
            retrieved_tables.append(rt)

        print("\n[Baseline 2] Using gold db_id, all tables in DB")
        print(f"  - db_id given    : {db_id}")
        print(f"  - target_db      : {target_db}")
        print(f"  - #tables in DB  : {len(table_names_for_db)}")

    print(f"  - allowed_tables ({len(allowed_tables)}) (full schema used):")
    for name in sorted(allowed_tables):
        print(f"    - {name}")

    # 3) 전체 스키마 DDL 생성 + 토큰 수 측정
    ddl_block, ddl_tokens = build_full_schema_context_from_schema_index(
        schema_db=schema_index_db,
        target_db_id=target_db,
        allowed_tables=allowed_tables,
        tokenizer=sql_tokenizer,
    )

    print("\n[Baseline 3] Full schema DDL stats")
    print(f"  - DDL tokens (approx, sql_tokenizer 기준): {ddl_tokens}")
    print(f"  - DDL length (chars): {len(ddl_block)}")

    # 질문/DDL 토큰 수 별도 계산
    try:
        q_tokens = len(sql_tokenizer.encode(question))
        ddl_tokens_re = len(sql_tokenizer.encode(ddl_block)) if ddl_block else 0
    except Exception:
        q_tokens = ddl_tokens_re = -1

    print("\n[Baseline 3-1] Final LLM input context token stats (approx.)")
    print(f"  - question_tokens      : {q_tokens}")
    print(f"  - schema_ctx_tokens(DDL): {ddl_tokens_re}")
    print(f"  - total_ctx_tokens_with_q: {q_tokens + ddl_tokens_re}")

    cell_context = ""  # 베이스라인은 셀 컨텍스트 사용 안 함

    print("\n[Baseline 3-2] Schema context preview")
    print("--------------------------------")
    print(ddl_block[:800] + ("...\n" if len(ddl_block) > 800 else "\n"))

    # 4) Text-to-SQL (Mistral-Nemo + 경량 self-correction)
    token_tracker.reset()

    agent = TextToSQLAgent(
        mode_prefix="baseline",
        extra_instructions=(
            "Use only the tables and columns from the schema summary. "
            "Do not invent new table or column names."
        ),
    )

    print("\n[Baseline 4] SQL generation (baseline, full schema)")
    print("[4-0] LLM main call input (high-level)")
    print(f"  - question      : {repr(question.strip())}")
    print(f"  - ddl_block len : {len(ddl_block)} chars")
    print(f"  - cell_context  : (not used, empty string)")
    print("  - extra_instructions:")
    print("    " + agent.extra_instructions)

    sql_result = agent.generate_sql(
        question=question.strip(),
        ddl_block=ddl_block,
        cell_context=cell_context,
        max_new_tokens=max_new_tokens_sql,
    )

    # 가능하다면 실제 프롬프트까지 출력
    print("\n[Baseline 4-1] (Optional) Actual prompts if exposed by agent/sql_result")
    if hasattr(sql_result, "prompt_main"):
        print("\n[Prompt - main SQL generation]")
        print("-" * 50)
        print(textwrap.indent(sql_result.prompt_main, prefix="    "))
    elif hasattr(agent, "last_main_prompt"):
        print("\n[Prompt - main SQL generation (agent.last_main_prompt)]")
        print("-" * 50)
        print(textwrap.indent(agent.last_main_prompt, prefix="    "))
    else:
        print("  - main prompt not exposed; consider adding debug fields in TextToSQLAgent.")

    if hasattr(sql_result, "prompt_selfcheck"):
        print("\n[Prompt - self-check]")
        print("-" * 50)
        print(textwrap.indent(sql_result.prompt_selfcheck, prefix="    "))
    elif hasattr(agent, "last_selfcheck_prompt"):
        print("\n[Prompt - self-check (agent.last_selfcheck_prompt)]")
        print("-" * 50)
        print(textwrap.indent(agent.last_selfcheck_prompt, prefix="    "))
    else:
        print("  - self-check prompt not exposed; "
              "add e.g. sql_result.prompt_selfcheck or agent.last_selfcheck_prompt for debugging.")

    print("\n[Baseline 4-2] SQL (draft)")
    print("--------------------------------")
    print(sql_result.sql_initial)

    print("\n[Baseline 4-3] SQL (final after self-correction)")
    print("--------------------------------")
    print(sql_result.sql)

    print("\n[Baseline 5] LLM token usage summary")
    print("--------------------------------")
    token_tracker.print_summary()

    print("\n[Baseline 5-1] token_tracker.data (raw)")
    print(token_tracker.data)

    return {
        "feats": feats,
        "retrieved_tables": retrieved_tables,
        "target_db": target_db,
        "allowed_tables": allowed_tables,
        "schema_context": ddl_block,
        "ddl_tokens": ddl_tokens,
        "sql_result": sql_result,
        "token_usage": token_tracker.data,
        "budgets": {
            "B_schema": B_schema,
            "B_cell": B_cell,
            "B_ctx": B_ctx,
            "T_max": T_max,
        },
    }



def run_baseline_full_schema_nologs(
    question: str,
    T_max: int = 2048,
    top_m_tables: int = 10,
    max_new_tokens_sql: int = 256,
    db_id: Optional[str] = None,   # ✅ gold dbid를 옵션으로 받음
):
    """
    베이스라인 파이프라인:
      - db_id가 None이면: 기존처럼 TableRetriever로 DB 선택
      - db_id가 주어지면: 해당 DB의 테이블들만 사용
    """

    # 1) 질문 분석 + 예산 (여기서는 참고용)
    feats = question_analyzer.analyze(question)
    B_schema, B_cell, B_ctx = token_budget_controller(feats, T_max=T_max)

    # 2) 테이블 / DB 선택
    if db_id is None:
        # 기존 로직: TableRetriever를 사용해 DB-first
        retrieved_tables = table_retriever.retrieve(feats.q_norm, top_m=top_m_tables)

        target_db = TableRetriever.pick_db_id(retrieved_tables)
        if target_db is None:
            raise RuntimeError("No target_db selected in baseline.")

        allowed_tables: Set[str] = {
            t.table_name_db for t in retrieved_tables if t.db_id == target_db
        }
    else:
        # db_id가 주어지면: 그 DB 안의 모든 테이블을 allowed_tables로 사용
        target_db = db_id

        # table_index_db에서 해당 DB의 모든 테이블명 수집
        table_names_for_db = sorted({
            row.table_name_db for row in table_index_db.rows
            if row.db_id == target_db
        })
        if not table_names_for_db:
            raise RuntimeError(f"[Baseline] No tables found for db_id={target_db} in table_index_db.")

        allowed_tables: Set[str] = set(table_names_for_db)

        # retrieved_tables 도 결과 dict에 넣을 수 있도록 대강 구성
        retrieved_tables = []
        for idx, row in enumerate(table_index_db.rows):
            if row.db_id != target_db:
                continue
            try:
                rt = RetrievedTable(
                    db_id=row.db_id,
                    table_name_db=row.table_name_db,
                    score=0.0,
                    table_idx=idx,
                    sim_table=0.0,
                    sim_col_max=0.0,
                    lex_score=0.0,
                )
            except TypeError:
                rt = RetrievedTable(
                    db_id=row.db_id,
                    table_name_db=row.table_name_db,
                    score=0.0,
                )
            retrieved_tables.append(rt)

    # 3) 전체 스키마 DDL 생성 + 토큰 수 측정
    ddl_block, ddl_tokens = build_full_schema_context_from_schema_index(
        schema_db=schema_index_db,
        target_db_id=target_db,
        allowed_tables=allowed_tables,
        tokenizer=sql_tokenizer,
    )
    # print(f"\n[Baseline] Full schema DDL tokens ≈ {ddl_tokens}")

    cell_context = ""  # 베이스라인은 셀 컨텍스트 사용 안 함

    # print("\n[Baseline] Schema context preview:")
    # print("--------------------------------")
    # print(ddl_block[:800], "...\n" if len(ddl_block) > 800 else "\n")

    # 4) Text-to-SQL (Mistral-Nemo + 경량 self-correction)
    token_tracker.reset()

    agent = TextToSQLAgent(
        mode_prefix="baseline",
        extra_instructions=(
            "Use only the tables and columns from the schema summary. "
            "Do not invent new table or column names."
        ),
    )

    sql_result = agent.generate_sql(
        question=question.strip(),
        ddl_block=ddl_block,
        cell_context=cell_context,
        max_new_tokens=max_new_tokens_sql,
    )


    return {
        "feats": feats,
        "retrieved_tables": retrieved_tables,
        "target_db": target_db,
        "allowed_tables": allowed_tables,
        "schema_context": ddl_block,
        "ddl_tokens": ddl_tokens,
        "sql_result": sql_result,
        "token_usage": token_tracker.data,
        "budgets": {
            "B_schema": B_schema,
            "B_cell": B_cell,
            "B_ctx": B_ctx,
        },
    }



In [ ]:
# ====== Cell 9': 전체 파이프라인 (고정 비율 버전, db_id 지원) ======
import re
from typing import List, Set, Dict, Any, Optional
import textwrap  # 긴 프롬프트 pretty print 용
import numpy as np


def _run_full_pipeline_fixed_ratio_impl(
    question: str,
    T_max: int = 600,
    schema_ratio: float = 0.6,         # ★ 고정 비율 (하이퍼파라미터)
    use_cell_gate: bool = True,        # False로 두면 항상 셀 사용
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    verbose: bool = True,
    db_id: Optional[str] = None,       # gold db_id 지원
) -> Dict[str, Any]:
    """
    고정 T_max + 고정 schema/cell 비율(static policy) 버전의 전체 파이프라인.
    - token_budget_controller를 사용하지 않고, T_max와 schema_ratio에서
      B_schema, B_cell, B_ctx를 직접 계산한다.
    - 이 버전을 여러 schema_ratio 값에 대해 dev set에서 돌려보고,
      best schema_ratio를 선택하면 "Static baseline" 완성.

    흐름:
      0) 질문 단위로 token_tracker 리셋
      1) QuestionAnalyzer → feats만 얻고, 예산은 수동 설정
      2) db_id에 따라 DB/테이블 검색
      3) SchemaRetriever / CellRetriever (LLM 기반 query expansion 사용)
      4) build_schema_context_hybrid / build_cell_context_from_selected_cells 로 압축 컨텍스트 생성
      5) TextToSQLAgent(mode_prefix="pipeline")로 main + 경량 self-check
      6) token_tracker 에서 토큰 사용량 수집
    """
    # ---------------- 0) 토큰 트래커 리셋 (질문당 1번) ----------------
    token_tracker.reset()

    # ---------------- 1) 질문 분석 + Static Budget ----------------
    feats = question_analyzer.analyze(question)

    # schema_ratio를 [0,1] 범위 안으로 클램프
    schema_ratio_clamped = max(0.0, min(1.0, float(schema_ratio)))
    B_ctx = int(T_max)                       # 컨텍스트 전체 예산 (schema+cell)
    B_schema = int(B_ctx * schema_ratio_clamped)
    B_cell = max(0, B_ctx - B_schema)

    if verbose:
        print("=" * 70)
        print("[Pipeline-Static] Question")
        print("- Raw :", repr(question))
        print("- Norm:", repr(feats.q_norm))
        print()
        print("[1] Question features")
        print(f"  - is_agg       : {feats.is_agg}")
        print(f"  - needs_join   : {feats.needs_join}")
        print(f"  - is_long      : {feats.is_long}")
        print(f"  - has_group_by : {feats.has_group_by}")
        print(f"  - needs_nesting: {feats.needs_nesting}")
        print()
        print(f"[1] Static token budgets (T_max={T_max}, schema_ratio={schema_ratio_clamped:.3f})")
        print(f"  - B_schema : {B_schema}")
        print(f"  - B_cell   : {B_cell}")
        print(f"  - B_ctx    : {B_ctx}  (B_schema + B_cell)")
        print(f"  - B_schema + B_cell : {B_schema + B_cell}")

    # ---------------- 2) 테이블 검색 ----------------
    if db_id is None:
        # 기존 로직: DB-first TableRetriever 사용
        retrieved_tables = table_retriever.retrieve(feats.q_norm, top_m=top_m_tables)
        target_db = TableRetriever.pick_db_id(retrieved_tables)
        if target_db is None:
            raise RuntimeError("No target_db selected in pipeline (static baseline).")
        allowed_tables: Set[str] = {
            t.table_name_db for t in retrieved_tables if t.db_id == target_db
        }
    else:
        # gold db_id가 주어졌으므로 해당 DB 안에서만 테이블 검색
        target_db = db_id

        # question → 임베딩
        q_emb = embed_texts([feats.q_norm])[0]   # (dim,)

        rows = table_index_db.rows
        emb  = table_index_db.emb

        # 주어진 db_id에 속하는 테이블들만 후보
        idx_list = [i for i, r in enumerate(rows) if r.db_id == target_db]
        if not idx_list:
            raise RuntimeError(f"No tables found in table_index_db for db_id={target_db}")

        sub_emb = emb[idx_list]
        sims = sub_emb @ q_emb
        order = np.argsort(-sims)[:top_m_tables]

        retrieved_tables = []
        for k in order:
            idx = idx_list[k]
            r = rows[idx]
            sim_table = float(sims[k])

            try:
                rt = RetrievedTable(
                    db_id=r.db_id,
                    table_name_db=r.table_name_db,
                    score=sim_table,
                    table_idx=idx,
                    sim_table=sim_table,
                    sim_col_max=0.0,
                    lex_score=0.0,
                )
            except TypeError:
                rt = RetrievedTable(
                    db_id=r.db_id,
                    table_name_db=r.table_name_db,
                    score=sim_table,
                )
            retrieved_tables.append(rt)

        allowed_tables = {t.table_name_db for t in retrieved_tables}

    if verbose:
        print("\n[2] Table retrieval (static baseline)")
        print(f"  - db_id given? : {db_id is not None}")
        print(f"  - target_db    : {target_db}")
        print(f"  - top_m_tables : {top_m_tables}")
        print("  - Retrieved tables:")
        for i, t in enumerate(retrieved_tables):
            print(f"    [{i:02d}] db={t.db_id:<20} table={t.table_name_db:<25} score={t.score:.4f}")
        print(f"  - allowed_tables ({len(allowed_tables)}): {sorted(allowed_tables)}")

    # ---------------- 3) 스키마 / 셀 Retrieval (LLM Q-exp 사용) ----------------
    selected_columns = schema_retriever.retrieve_for_db_tables(
        question=feats.q_raw,
        target_db_id=target_db,
        allowed_tables=allowed_tables,
        top_k=top_k_cols,
    )

    if use_cell_gate:
        use_cell = should_use_cell_context(feats, question)
    else:
        use_cell = True

    B_cell_eff = B_cell if use_cell else 0

    selected_cells: List[SelectedCell] = []
    if B_cell_eff > 0:
        selected_cells = cell_retriever.retrieve_for_db_tables(
            question=feats.q_raw,
            target_db_id=target_db,
            allowed_tables=allowed_tables,
            top_k=top_k_cells,
        )

    if verbose:
        print("\n[3] Schema / Cell retrieval (static baseline, after LLM query expansion)")
        print(f"  - top_k_cols        : {top_k_cols}")
        print(f"  - #selected_columns : {len(selected_columns)}")
        for i, col in enumerate(selected_columns[:10]):
            print(f"    [col {i:02d}] {col}")
        if len(selected_columns) > 10:
            print(f"    ... (+{len(selected_columns) - 10} more columns)")

        print()
        print(f"  - use_cell_gate            : {use_cell_gate}")
        print(f"  - should_use_cell_context  : {use_cell}")
        print(f"  - B_cell_eff (actual B_cell used) : {B_cell_eff}")
        print(f"  - top_k_cells         : {top_k_cells}")
        print(f"  - #selected_cells     : {len(selected_cells)}")
        for i, cell in enumerate(selected_cells[:10]):
            print(f"    [cell {i:02d}] {cell}")
        if len(selected_cells) > 10:
            print(f"    ... (+{len(selected_cells) - 10} more cells)")

    # ---------------- 4) 스키마 / 셀 컨텍스트 생성 (압축) ----------------
    schema_context, schema_tokens_used = build_schema_context_hybrid(
        schema_db=schema_index_db,
        target_db_id=target_db,
        allowed_tables=allowed_tables,
        selected_columns=selected_columns,
        B_schema=B_schema,
        tokenizer=sql_tokenizer,
    )

    cell_context, cell_tokens_used = build_cell_context_from_selected_cells(
        selected_cells=selected_cells,
        B_cell=B_cell_eff,
        tokenizer=sql_tokenizer,
    )

    ctx_tokens_total = schema_tokens_used + cell_tokens_used

    if verbose:
        print("\n[4] Compressed schema / cell context (static baseline)")
        print(f"  - schema_tokens_used : {schema_tokens_used} / B_schema={B_schema}")
        print(f"  - cell_tokens_used   : {cell_tokens_used} / B_cell_eff={B_cell_eff}")
        print(f"  - total_ctx_tokens   : {ctx_tokens_total} (schema+cell)")
        print(f"  - B_ctx (static)     : {B_ctx}")

        try:
            q_tokens = len(sql_tokenizer.encode(question))
            ddl_tokens = len(sql_tokenizer.encode(schema_context)) if schema_context else 0
            cell_tokens_reenc = len(sql_tokenizer.encode(cell_context)) if cell_context else 0
        except Exception:
            q_tokens = ddl_tokens = cell_tokens_reenc = -1

        print("\n[4-1] Final LLM input context token stats (approx. sql_tokenizer 기준)")
        print(f"  - question_tokens           : {q_tokens}")
        print(f"  - schema_ctx_tokens(re-enc) : {ddl_tokens}")
        print(f"  - cell_ctx_tokens  (re-enc) : {cell_tokens_reenc}")
        print(f"  - total_ctx_tokens_with_q   : {q_tokens + ddl_tokens + cell_tokens_reenc}")

        print("\n[Schema context preview]")
        print("-" * 40)
        if schema_context:
            print(schema_context[:800] + ("...\n" if len(schema_context) > 800 else "\n"))
        else:
            print("(empty)\n")

        print("[Cell context preview]")
        print("-" * 40)
        if cell_context:
            print(cell_context[:800] + ("...\n" if len(cell_context) > 800 else "\n"))
        else:
            print("(empty)\n")

    # ---------------- 5) Text-to-SQL (main + 경량 self-check) ----------------
    agent = TextToSQLAgent(
        mode_prefix="pipeline",
        extra_instructions=(
            "Use only the tables and columns from the schema summary. "
            "Do not invent new table or column names. "
            "Make sure numeric conditions match the question."
        ),
    )

    if verbose:
        print("\n[5] SQL generation (pipeline / static baseline)")
        print("[5-0] LLM main call input (high-level)")
        print(f"  - question     : {repr(question.strip())}")
        print(f"  - ddl_block len: {len(schema_context)} chars")
        print(f"  - cell_ctx len : {len(cell_context)} chars")
        print("  - extra_instructions:")
        print("    " + agent.extra_instructions)

    sql_result = agent.generate_sql(
        question=question.strip(),
        ddl_block=schema_context,
        cell_context=cell_context,
        max_new_tokens=max_new_tokens_sql,
    )

    if verbose:
        print("\n[5-0-1] (Optional) Actual prompts if exposed by agent/sql_result")
        if hasattr(sql_result, "prompt_main"):
            print("\n[Prompt - main SQL generation]")
            print("-" * 50)
            print(textwrap.indent(sql_result.prompt_main, prefix="    "))
        elif hasattr(agent, "last_main_prompt"):
            print("\n[Prompt - main SQL generation (agent.last_main_prompt)]")
            print("-" * 50)
            print(textwrap.indent(agent.last_main_prompt, prefix="    "))
        else:
            print("  - main prompt not exposed; consider adding debug fields in TextToSQLAgent.")

        if hasattr(sql_result, "prompt_selfcheck"):
            print("\n[Prompt - self-check]")
            print("-" * 50)
            print(textwrap.indent(sql_result.prompt_selfcheck, prefix="    "))
        elif hasattr(agent, "last_selfcheck_prompt"):
            print("\n[Prompt - self-check (agent.last_selfcheck_prompt)]")
            print("-" * 50)
            print(textwrap.indent(agent.last_selfcheck_prompt, prefix="    "))
        else:
            print("  - self-check prompt not exposed; "
                  "add e.g. sql_result.prompt_selfcheck or agent.last_selfcheck_prompt for debugging.")

        print("\n[5-1] Initial SQL (draft):")
        print("-" * 40)
        print(sql_result.sql_initial)

        print("\n[5-2] Final SQL after self-correction:")
        print("-" * 40)
        print(sql_result.sql)

        print("\n[6] LLM token usage summary (pipeline / static baseline)")
        print("-" * 40)
        token_tracker.print_summary()

        print("\n[6-1] token_tracker.data (raw)")
        print(token_tracker.data)

    # ---------------- 6) 결과 패키징 ----------------
    result = {
        "feats": feats,
        "retrieved_tables": retrieved_tables,
        "target_db": target_db,
        "allowed_tables": allowed_tables,
        "selected_columns": selected_columns,
        "selected_cells": selected_cells,
        "schema_context": schema_context,
        "cell_context": cell_context,
        "sql_result": sql_result,
        "token_usage": token_tracker.data,
        "budgets": {
            "B_schema": B_schema,
            "B_cell": B_cell,
            "B_ctx": B_ctx,
            "T_max": T_max,
            "schema_ratio": schema_ratio_clamped,
        },
        "ctx_tokens_used": {
            "schema": schema_tokens_used,
            "cell": cell_tokens_used,
            "total": ctx_tokens_total,
        },
    }
    return result


def run_full_pipeline_fixed_ratio(
    question: str,
    T_max: int = 500,
    schema_ratio: float = 0.6,
    use_cell_gate: bool = True,
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    db_id: Optional[str] = None,
):
    """
    고정 T_max + 고정 schema_ratio (static policy) + 로그 출력 버전.
    dev set에서 이 함수를 여러 schema_ratio 값으로 돌려보고
    best schema_ratio를 선택하면 Static baseline 완성.
    """
    return _run_full_pipeline_fixed_ratio_impl(
        question=question,
        T_max=T_max,
        schema_ratio=schema_ratio,
        use_cell_gate=use_cell_gate,
        top_m_tables=top_m_tables,
        top_k_cols=top_k_cols,
        top_k_cells=top_k_cells,
        max_new_tokens_sql=max_new_tokens_sql,
        verbose=True,
        db_id=db_id,
    )


def run_full_pipeline_fixed_ratio_nologs(
    question: str,
    T_max: int = 500,
    schema_ratio: float = 0.6,
    use_cell_gate: bool = True,
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    db_id: Optional[str] = None,
):
    """
    평가/배치 실행용 조용한 버전.
    Static baseline (고정 비율) 평가 시 Spider evaluator에서 이 함수를 래핑해서 사용.
    """
    return _run_full_pipeline_fixed_ratio_impl(
        question=question,
        T_max=T_max,
        schema_ratio=schema_ratio,
        use_cell_gate=use_cell_gate,
        top_m_tables=top_m_tables,
        top_k_cols=top_k_cols,
        top_k_cells=top_k_cells,
        max_new_tokens_sql=max_new_tokens_sql,
        verbose=False,
        db_id=db_id,
    )


In [ ]:
# ===== 새로 추가: T_max / 비율 없이 top-k만 사용하는 파이프라인 =====
def _run_full_pipeline_topk_only_impl(
    question: str,
    use_cell_gate: bool = True,
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    verbose: bool = True,
    db_id: Optional[str] = None,
) -> Dict[str, Any]:
    """
    🔹 T_max / schema_ratio 전혀 쓰지 않고
    🔹 스키마/셀 컨텍스트 크기는
        - top_m_tables, top_k_cols, top_k_cells
      만으로 제한하는 베이스라인.

    build_schema_context_hybrid / build_cell_context_from_selected_cells 에는
    매우 큰 B_schema, B_cell 을 넘겨서 사실상 "no limit"으로 사용하고,
    실제 제한은 retrieval 단계(top_k)에서만 걸린다.
    """
    # ---------------- 0) 토큰 트래커 리셋 ----------------
    token_tracker.reset()

    # ---------------- 1) 질문 분석 (예산 계산은 안 함) ----------------
    feats = question_analyzer.analyze(question)

    # 예산은 고정값 대신 "무제한"으로 두고, top-k만 사용
    # (build_*_context 함수가 B_* 이하로만 자르므로, 충분히 큰 값으로 설정)
    B_schema = 10**9
    B_cell_max = 10**9   # 셀 컨텍스트는 gate에 따라 0 또는 큰 값

    if verbose:
        print("=" * 70)
        print("[Pipeline-TopKOnly] Question")
        print("- Raw :", repr(question))
        print("- Norm:", repr(feats.q_norm))
        print()
        print("[1] Token budget: disabled (top-k only)")
        print(f"  - top_m_tables : {top_m_tables}")
        print(f"  - top_k_cols   : {top_k_cols}")
        print(f"  - top_k_cells  : {top_k_cells}")
        print(f"  - B_schema(no limit-ish) : {B_schema}")
        print(f"  - B_cell_max(no limit-ish): {B_cell_max}")

    # ---------------- 2) 테이블 검색 (원래 코드 그대로) ----------------
    if db_id is None:
        retrieved_tables = table_retriever.retrieve(feats.q_norm, top_m=top_m_tables)
        target_db = TableRetriever.pick_db_id(retrieved_tables)
        if target_db is None:
            raise RuntimeError("No target_db selected in pipeline (topk-only).")
        allowed_tables: Set[str] = {
            t.table_name_db for t in retrieved_tables if t.db_id == target_db
        }
    else:
        target_db = db_id

        q_emb = embed_texts([feats.q_norm])[0]   # (dim,)
        rows = table_index_db.rows
        emb  = table_index_db.emb

        idx_list = [i for i, r in enumerate(rows) if r.db_id == target_db]
        if not idx_list:
            raise RuntimeError(f"No tables found in table_index_db for db_id={target_db}")

        sub_emb = emb[idx_list]
        sims = sub_emb @ q_emb
        order = np.argsort(-sims)[:top_m_tables]

        retrieved_tables = []
        for k in order:
            idx = idx_list[k]
            r = rows[idx]
            sim_table = float(sims[k])
            try:
                rt = RetrievedTable(
                    db_id=r.db_id,
                    table_name_db=r.table_name_db,
                    score=sim_table,
                    table_idx=idx,
                    sim_table=sim_table,
                    sim_col_max=0.0,
                    lex_score=0.0,
                )
            except TypeError:
                rt = RetrievedTable(
                    db_id=r.db_id,
                    table_name_db=r.table_name_db,
                    score=sim_table,
                )
            retrieved_tables.append(rt)

        allowed_tables = {t.table_name_db for t in retrieved_tables}

    if verbose:
        print("\n[2] Table retrieval (topk-only)")
        print(f"  - db_id given? : {db_id is not None}")
        print(f"  - target_db    : {target_db}")
        print(f"  - top_m_tables : {top_m_tables}")
        print("  - Retrieved tables:")
        for i, t in enumerate(retrieved_tables):
            print(f"    [{i:02d}] db={t.db_id:<20} table={t.table_name_db:<25} score={t.score:.4f}")
        print(f"  - allowed_tables ({len(allowed_tables)}): {sorted(allowed_tables)}")

    # ---------------- 3) 스키마 / 셀 Retrieval (LLM Q-exp + top_k만 사용) ----------------
    selected_columns = schema_retriever.retrieve_for_db_tables(
        question=feats.q_raw,
        target_db_id=target_db,
        allowed_tables=allowed_tables,
        top_k=top_k_cols,
    )

    if use_cell_gate:
        use_cell = should_use_cell_context(feats, question)
    else:
        use_cell = True

    B_cell_eff = B_cell_max if use_cell else 0

    selected_cells: List[SelectedCell] = []
    if B_cell_eff > 0:
        selected_cells = cell_retriever.retrieve_for_db_tables(
            question=feats.q_raw,
            target_db_id=target_db,
            allowed_tables=allowed_tables,
            top_k=top_k_cells,
        )

    if verbose:
        print("\n[3] Schema / Cell retrieval (topk-only)")
        print(f"  - top_k_cols        : {top_k_cols}")
        print(f"  - #selected_columns : {len(selected_columns)}")
        for i, col in enumerate(selected_columns[:10]):
            print(f"    [col {i:02d}] {col}")
        if len(selected_columns) > 10:
            print(f"    ... (+{len(selected_columns) - 10} more columns)")

        print()
        print(f"  - use_cell_gate            : {use_cell_gate}")
        print(f"  - should_use_cell_context  : {use_cell}")
        print(f"  - B_cell_eff (max cell tokens, effectively no limit): {B_cell_eff}")
        print(f"  - top_k_cells         : {top_k_cells}")
        print(f"  - #selected_cells     : {len(selected_cells)}")
        for i, cell in enumerate(selected_cells[:10]):
            print(f"    [cell {i:02d}] {cell}")
        if len(selected_cells) > 10:
            print(f"    ... (+{len(selected_cells) - 10} more cells)")

    # ---------------- 4) 스키마 / 셀 컨텍스트 생성 ----------------
    schema_context, schema_tokens_used = build_schema_context_hybrid(
        schema_db=schema_index_db,
        target_db_id=target_db,
        allowed_tables=allowed_tables,
        selected_columns=selected_columns,
        B_schema=B_schema,              # 🔹 매우 큰 값 → 사실상 top_k만 영향
        tokenizer=sql_tokenizer,
    )

    cell_context, cell_tokens_used = build_cell_context_from_selected_cells(
        selected_cells=selected_cells,
        B_cell=B_cell_eff,              # 🔹 매우 큰 값 → 사실상 top_k만 영향
        tokenizer=sql_tokenizer,
    )

    ctx_tokens_total = schema_tokens_used + cell_tokens_used

    if verbose:
        print("\n[4] Compressed schema / cell context (topk-only)")
        print(f"  - schema_tokens_used : {schema_tokens_used}")
        print(f"  - cell_tokens_used   : {cell_tokens_used}")
        print(f"  - total_ctx_tokens   : {ctx_tokens_total} (schema+cell)")

        try:
            q_tokens = len(sql_tokenizer.encode(question))
            ddl_tokens = len(sql_tokenizer.encode(schema_context)) if schema_context else 0
            cell_tokens_reenc = len(sql_tokenizer.encode(cell_context)) if cell_context else 0
        except Exception:
             q_tokens = ddl_tokens = cell_tokens_reenc = -1

        print("\n[4-1] Final LLM input context token stats (approx.)")
        print(f"  - question_tokens           : {q_tokens}")
        print(f"  - schema_ctx_tokens(re-enc) : {ddl_tokens}")
        print(f"  - cell_ctx_tokens  (re-enc) : {cell_tokens_reenc}")
        print(f"  - total_ctx_tokens_with_q   : {q_tokens + ddl_tokens + cell_tokens_reenc}")

    # ---------------- 5) Text-to-SQL (원래 Agent 그대로) ----------------
    agent = TextToSQLAgent(
        mode_prefix="pipeline",
        extra_instructions=(
            "Use only the tables and columns from the schema summary. "
            "Do not invent new table or column names. "
            "Make sure numeric conditions match the question."
        ),
    )

    sql_result = agent.generate_sql(
        question=question.strip(),
        ddl_block=schema_context,
        cell_context=cell_context,
        max_new_tokens=max_new_tokens_sql,
    )

    if verbose:
        print("\n[5] Final SQL (topk-only)")
        print("-" * 40)
        print(sql_result.sql)

        print("\n[6] LLM token usage summary (pipeline / topk-only)")
        print("-" * 40)
        token_tracker.print_summary()
        print("\n[6-1] token_tracker.data (raw)")
        print(token_tracker.data)

    # ---------------- 6) 결과 패키징 ----------------
    result = {
        "feats": feats,
        "retrieved_tables": retrieved_tables,
        "target_db": target_db,
        "allowed_tables": allowed_tables,
        "selected_columns": selected_columns,
        "selected_cells": selected_cells,
        "schema_context": schema_context,
        "cell_context": cell_context,
        "sql_result": sql_result,
        "token_usage": token_tracker.data,
        "budgets": {
            # 이제 T_max 기반 예산은 없고, 실제 사용량만 기록
            "B_schema": schema_tokens_used,
            "B_cell": cell_tokens_used,
            "B_ctx": ctx_tokens_total,
            "T_max": None,
            "schema_ratio": None,
            "mode": "topk_only",
        },
        "ctx_tokens_used": {
            "schema": schema_tokens_used,
            "cell": cell_tokens_used,
            "total": ctx_tokens_total,
        },
    }
    return result


def run_full_pipeline_topk_only(
    question: str,
    use_cell_gate: bool = True,
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    db_id: Optional[str] = None,
    verbose: bool = True,
) -> Dict[str, Any]:
    return _run_full_pipeline_topk_only_impl(
        question=question,
        use_cell_gate=use_cell_gate,
        top_m_tables=top_m_tables,
        top_k_cols=top_k_cols,
        top_k_cells=top_k_cells,
        max_new_tokens_sql=max_new_tokens_sql,
        verbose=verbose,
        db_id=db_id,
    )


def run_full_pipeline_topk_only_nologs(
    question: str,
    use_cell_gate: bool = True,
    top_m_tables: int = 10,
    top_k_cols: int = 50,
    top_k_cells: int = 50,
    max_new_tokens_sql: int = 256,
    db_id: Optional[str] = None,
) -> Dict[str, Any]:
    return _run_full_pipeline_topk_only_impl(
        question=question,
        use_cell_gate=use_cell_gate,
        top_m_tables=top_m_tables,
        top_k_cols=top_k_cols,
        top_k_cells=top_k_cells,
        max_new_tokens_sql=max_new_tokens_sql,
        verbose=False,
        db_id=db_id,
    )


---
### MMQA 데이터셋 테스트

In [ ]:
!pip install -q nltk rouge-score

  Preparing metadata (setup.py) ... done


In [ ]:
import os
import json
import sqlite3
from typing import Dict, Any, List, Tuple
from collections import defaultdict

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm  # 진행률 표시

# ===== 경로 설정 =====
MMQA_DIR = "/content/drive/MyDrive/ai_intensive2/MMQA"
SPIDER_DB_ROOT = "/content/drive/MyDrive/ai_intensive2/spider_data/database"
debug_printed = False  # token_usage 구조를 한 번만 찍기 위한 플래그


def get_db_path(db_id: str) -> str:
    """
    Spider/MMQA DB 구조:
      spider_data/database/{db_id}/{db_id}.sqlite
    """
    return os.path.join(SPIDER_DB_ROOT, db_id, f"{db_id}.sqlite")


# ===== DB 실행 / 결과 canonicalization =====

def execute_sql_safe(conn: sqlite3.Connection, sql: str) -> Tuple[bool, List[Tuple[Any, ...]]]:
    """
    SQL을 실행하고 (성공 여부, rows) 를 반환.
    실패 시 (False, []).

    conn: sqlite3.Connection
    sql : 실행할 SQL 문자열
    """
    try:
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        return True, rows
    except Exception:
        # 디버깅이 필요하면 여기서 print(e) 등을 추가
        return False, []


def canonicalize_rows(rows: List[Tuple[Any, ...]]) -> Tuple[List[Tuple[str, ...]], str]:
    """
    - 각 cell을 str로 바꾸고
    - row들을 정렬
    - "\n".join("\t".join(row)) 형태의 canonical string 반환
    """
    rows_str = [tuple("" if (v is None) else str(v) for v in row) for row in rows]
    rows_sorted = sorted(rows_str)
    text = "\n".join(["\t".join(r) for r in rows_sorted])
    return rows_sorted, text


# ===== Metric 계산 =====

_smooth = SmoothingFunction().method1
_rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)


def compute_em(gold_text: str, pred_text: str) -> float:
    """Exact Match: 결과 텍스트가 완전히 동일하면 1, 아니면 0."""
    return 1.0 if gold_text == pred_text else 0.0


def compute_pm_row_f1(gold_rows: List[Tuple[str, ...]], pred_rows: List[Tuple[str, ...]]) -> float:
    """
    Partial Match: row-level F1
    - 각 row를 tuple(str, ...) 로 보고 set으로 비교
    """
    gold_set = set(gold_rows)
    pred_set = set(pred_rows)

    if not gold_set and not pred_set:
        return 1.0
    if not pred_set and gold_set:
        return 0.0

    inter = len(gold_set & pred_set)
    if inter == 0:
        return 0.0

    precision = inter / len(pred_set) if pred_set else 0.0
    recall = inter / len(gold_set) if gold_set else 0.0
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)


def compute_rouge_bleu(gold_text: str, pred_text: str) -> Dict[str, float]:
    """
    gold/pred 결과 문자열을 가지고 ROUGE-1 / ROUGE-L / BLEU 계산.
    """
    if not gold_text and not pred_text:
        return {"rouge1": 1.0, "rougeL": 1.0, "bleu": 1.0}
    if not pred_text and gold_text:
        return {"rouge1": 0.0, "rougeL": 0.0, "bleu": 0.0}

    # ROUGE
    rouge_scores = _rouge.score(gold_text, pred_text)
    rouge1_f = rouge_scores["rouge1"].fmeasure
    rougeL_f = rouge_scores["rougeL"].fmeasure

    # BLEU (1~4-gram sentence BLEU)
    ref_tokens = gold_text.split()
    pred_tokens = pred_text.split()
    if len(pred_tokens) == 0:
        bleu = 0.0
    else:
        bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=_smooth)

    return {"rouge1": rouge1_f, "rougeL": rougeL_f, "bleu": bleu}


# ===== 토큰 사용량 추출 (쿼리 확장 / 메인 SQL / self-check 별로 분리) =====

def extract_stage_token_stats(token_usage: Dict[str, Any]) -> Dict[str, int]:
    """
    DualTokenTracker.data 구조를 받아서
    - 쿼리 확장 LLM (qexp)
    - 메인 SQL 생성 LLM (sql_main)
    - self-check LLM (sql_selfcheck)
    별로 input/output/total 토큰 사용량을 집계한다.

    token_usage 예시:
      {
        "pipeline_qexp":          {"input": 120, "output": 30, "total": 150},
        "pipeline_sql_main":      {"input": 900, "output": 150, "total": 1050},
        "pipeline_sql_selfcheck": {"input": 880, "output": 40,  "total": 920},
        ...
      }
    """

    out = {
        "qexp_input_tokens": 0,
        "qexp_output_tokens": 0,
        "qexp_total_tokens": 0,
        "sql_main_input_tokens": 0,
        "sql_main_output_tokens": 0,
        "sql_main_total_tokens": 0,
        "sql_selfcheck_input_tokens": 0,
        "sql_selfcheck_output_tokens": 0,
        "sql_selfcheck_total_tokens": 0,
    }

    if not token_usage or not isinstance(token_usage, dict):
        return out

    def map_stage(name: str) -> str:
        """
        mode 이름을 어느 스테이지(qexp / sql_main / sql_selfcheck)에 매핑할지 결정.
        - 쿼리 확장:  qexp, queryexp 등
        - SQL main:  sql + (self/check 없을 때)
        - self-check: sql + self / check 등이 함께 있을 때
        """
        n = name.lower()

        # 쿼리 확장
        if any(x in n for x in ["qexp", "queryexp"]):
            return "qexp"

        # self-check (sql + self/check 등의 조합)
        if ("sql" in n) and any(x in n for x in ["self", "check", "verify", "validate"]):
            return "sql_selfcheck"

        # 나머지 sql 관련은 main 으로
        if "sql" in n:
            return "sql_main"

        # 그 외는 지금은 무시 (필요하면 나중에 stage 추가)
        return ""

    for mode_name, stats in token_usage.items():
        if not isinstance(stats, dict):
            continue

        stage = map_stage(mode_name)
        if not stage:
            continue

        # DualTokenTracker: {"input": .., "output": .., "total": ..}
        # 혹시 prompt_tokens / completion_tokens / total_tokens 로 들어올 수도 있어서 fallback 지원
        inp = stats.get("input", stats.get("prompt_tokens", 0)) or 0
        outp = stats.get("output", stats.get("completion_tokens", 0)) or 0
        total = stats.get("total", stats.get("total_tokens", inp + outp)) or 0

        try:
            inp = int(inp)
        except Exception:
            inp = 0
        try:
            outp = int(outp)
        except Exception:
            outp = 0
        try:
            total = int(total)
        except Exception:
            total = inp + outp

        out[f"{stage}_input_tokens"] += inp
        out[f"{stage}_output_tokens"] += outp
        out[f"{stage}_total_tokens"] += total

    return out


# ===== 메인 평가 루프 =====

def evaluate_mmqa(
    json_path: str,
    output_prefix: str,
    run_fn,  # 예: run_baseline_full_schema_nologs 또는 run_full_pipeline_nologs
):
    """
    MMQA 테스트 json 파일을 읽어서:
      - run_fn(question=..., db_id=...) 으로 SQL 생성
      - gold / pred SQL을 sqlite에 실행
      - EM / PM / ROUGE-1 / ROUGE-L / BLEU 계산
      - per-example 결과 + 토큰 사용량(원본 + stage별)을 JSON으로 저장
      - 전체 평균 metric/토큰 사용량을 report JSON으로 저장

    run_fn은 다음 시그니처를 가져야 한다:
        run_fn(question: str, db_id: Optional[str]) -> Dict[str, Any]

    반환 dict 안에는 최소한 다음 키가 있어야 한다:
        - "sql_result"  (sql_result.sql 에 최종 SQL이 있다고 가정)
        - "token_usage" (token_tracker.data 형태, mode별 토큰 사용량)
    """
    global debug_printed

    print("=" * 80)
    print(f"[EVAL] Loading dataset from: {json_path}")

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"[EVAL] #examples: {len(data)}")

    # ✅ 쿼리 확장 캐시 초기화 (반복 실험에서도 qexp 토큰 정상 측정)
    if "query_expander" in globals():
        qe = globals()["query_expander"]
        if hasattr(qe, "cache"):
            qe.cache.clear()
            print("[EVAL] Cleared query_expander.cache for fresh token counting.")

    # DB 연결 캐시
    conn_cache: Dict[str, sqlite3.Connection] = {}

    def get_conn(db_id: str) -> sqlite3.Connection:
        if db_id not in conn_cache:
            db_path = get_db_path(db_id)
            if not os.path.exists(db_path):
                raise FileNotFoundError(f"DB not found for db_id={db_id}: {db_path}")
            conn_cache[db_id] = sqlite3.connect(db_path)
        return conn_cache[db_id]

    results_per_example: List[Dict[str, Any]] = []

    # 누적 통계
    sum_metrics = defaultdict(float)
    sum_tokens = defaultdict(float)
    n_examples = 0

    for ex in tqdm(
        data,
        desc=f"Evaluating {os.path.basename(json_path)}",
        ncols=80,
    ):
        ex_id = ex.get("id")
        db_id = ex.get("db_id")
        question = ex.get("question", "")
        gold_sql = ex.get("sql", "")

        # 1) SQL 생성 (baseline 또는 pipeline, run_fn에 따라 달라짐)
        try:
            out = run_fn(
                question=question,
                db_id=db_id,
            )
            sql_result = out.get("sql_result", None)
            pred_sql = getattr(sql_result, "sql", "") if sql_result is not None else ""
            token_usage = out.get("token_usage", {}) or {}

            # 디버깅: 맨 처음 한 번만 raw token_usage 구조를 찍기
            if (not debug_printed) and token_usage:
                print("\n[DEBUG] token_usage sample:")
                import pprint
                pprint.pprint(token_usage)
                debug_printed = True

        except Exception as e:
            print(f"\n[WARN] LLM error on example id={ex_id}, db_id={db_id}: {e}")
            pred_sql = ""
            token_usage = {}

        # 2) gold / pred SQL 실행
        gold_rows, pred_rows = [], []
        gold_text, pred_text = "", ""

        try:
            conn = get_conn(db_id)
        except Exception as e:
            print(f"\n[ERROR] DB connection error for db_id={db_id}: {e}")
            conn = None

        if conn is not None:
            # gold
            ok_g, rows_g = execute_sql_safe(conn, gold_sql) if gold_sql else (False, [])
            if ok_g:
                gold_rows, gold_text = canonicalize_rows(rows_g)
            else:
                gold_rows, gold_text = [], ""

            # pred
            ok_p, rows_p = execute_sql_safe(conn, pred_sql) if pred_sql else (False, [])
            if ok_p:
                pred_rows, pred_text = canonicalize_rows(rows_p)
            else:
                pred_rows, pred_text = [], ""

        # 3) metric 계산
        em = compute_em(gold_text, pred_text)
        pm = compute_pm_row_f1(gold_rows, pred_rows)
        rouge_bleu = compute_rouge_bleu(gold_text, pred_text)

        # 4) 토큰 사용량 (스테이지별 집계)
        stage_tokens = extract_stage_token_stats(token_usage)

        # per-example 결과 저장
        ex_result: Dict[str, Any] = {
            "id": ex_id,
            "db_id": db_id,
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "gold_result_text": gold_text,
            "pred_result_text": pred_text,
            "metrics": {
                "EM": em,
                "PM": pm,
                "ROUGE-1": rouge_bleu["rouge1"],
                "ROUGE-L": rouge_bleu["rougeL"],
                "BLEU": rouge_bleu["bleu"],
            },
            # DualTokenTracker.data 전체 (mode별 input/output/total)
            "token_usage_raw": token_usage,
        }
        # 스테이지별 input/output/total 토큰 수를 flat하게 기록
        ex_result.update(stage_tokens)

        results_per_example.append(ex_result)

        # 누적
        sum_metrics["EM"] += em
        sum_metrics["PM"] += pm
        sum_metrics["ROUGE-1"] += rouge_bleu["rouge1"]
        sum_metrics["ROUGE-L"] += rouge_bleu["rougeL"]
        sum_metrics["BLEU"] += rouge_bleu["bleu"]

        for k, v in stage_tokens.items():
            sum_tokens[k] += v

        n_examples += 1

    # DB 연결 닫기
    for conn in conn_cache.values():
        conn.close()

    # 전체 평균/총합 report
    if n_examples > 0:
        avg_metrics = {k: v / n_examples for k, v in sum_metrics.items()}
        avg_tokens = {k: v / n_examples for k, v in sum_tokens.items()}
    else:
        avg_metrics = {}
        avg_tokens = {}

    # 스테이지별 total + 전체 total
    total_tokens = {
        "qexp_input_tokens": sum_tokens.get("qexp_input_tokens", 0.0),
        "qexp_output_tokens": sum_tokens.get("qexp_output_tokens", 0.0),
        "qexp_total_tokens": sum_tokens.get("qexp_total_tokens", 0.0),
        "sql_main_input_tokens": sum_tokens.get("sql_main_input_tokens", 0.0),
        "sql_main_output_tokens": sum_tokens.get("sql_main_output_tokens", 0.0),
        "sql_main_total_tokens": sum_tokens.get("sql_main_total_tokens", 0.0),
        "sql_selfcheck_input_tokens": sum_tokens.get("sql_selfcheck_input_tokens", 0.0),
        "sql_selfcheck_output_tokens": sum_tokens.get("sql_selfcheck_output_tokens", 0.0),
        "sql_selfcheck_total_tokens": sum_tokens.get("sql_selfcheck_total_tokens", 0.0),
    }
    total_tokens["all_total_tokens"] = (
        total_tokens["qexp_total_tokens"]
        + total_tokens["sql_main_total_tokens"]
        + total_tokens["sql_selfcheck_total_tokens"]
    )

    report = {
        "dataset_path": json_path,
        "num_examples": n_examples,
        "avg_metrics": avg_metrics,
        "avg_tokens": avg_tokens,
        "total_tokens": total_tokens,
    }

    # 저장 경로
    result_path = os.path.join(MMQA_DIR, f"{output_prefix}_results.json")
    report_path = os.path.join(MMQA_DIR, f"{output_prefix}_report.json")

    with open(result_path, "w", encoding="utf-8") as f:
        json.dump(results_per_example, f, ensure_ascii=False, indent=2)

    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)

    print(f"\n[SAVE] Per-example results saved to: {result_path}")
    print(f"[SAVE] Summary report saved to      : {report_path}")


In [ ]:
from functools import partial

# ===== 고정 설정 =====
T_max_for_ratio_search = 384          # 토큰 예산 고정
top_k_cols_fixed = 24                 # 컬럼 top-k 고정
top_k_cells_fixed = 48                # 셀 top-k 고정

mmqa2_test_path = os.path.join(MMQA_DIR, "mmqa2_test.json")
mmqa3_test_path = os.path.join(MMQA_DIR, "mmqa3_test.json")


run_pipeline_topk_only = partial(
    run_full_pipeline_topk_only_nologs,
    use_cell_gate=True,
    top_m_tables=10,
    top_k_cols=top_k_cols_fixed,    # 기존 static 과 동일한 값 쓰면 비교 가능
    top_k_cells=top_k_cells_fixed,
    max_new_tokens_sql=256,
)

evaluate_mmqa(
    json_path=mmqa2_test_path,
    output_prefix="mmqa2_test_topk_only",
    run_fn=run_pipeline_topk_only,
)



[EVAL] Loading dataset from: /content/drive/MyDrive/ai_intensive2/MMQA/mmqa2_test.json
[EVAL] #examples: 2074
[EVAL] Cleared query_expander.cache for fresh token counting.


Evaluating mmqa2_test.json: 100%|█████████| 2074/2074 [5:32:24<00:00,  9.62s/it]



[SAVE] Per-example results saved to: /content/drive/MyDrive/ai_intensive2/MMQA/mmqa2_test_topk_only_results.json
[SAVE] Summary report saved to      : /content/drive/MyDrive/ai_intensive2/MMQA/mmqa2_test_topk_only_report.json


In [ ]:
from functools import partial

# ===== 고정 설정 =====
T_max_for_ratio_search = 384          # 토큰 예산 고정
top_k_cols_fixed = 24                 # 컬럼 top-k 고정
top_k_cells_fixed = 48                # 셀 top-k 고정

mmqa2_test_path = os.path.join(MMQA_DIR, "mmqa2_test_10.json")
mmqa3_test_path = os.path.join(MMQA_DIR, "mmqa3_test.json")


run_pipeline_topk_only = partial(
    run_full_pipeline_topk_only_nologs,
    use_cell_gate=True,
    top_m_tables=10,
    top_k_cols=top_k_cols_fixed,    # 기존 static 과 동일한 값 쓰면 비교 가능
    top_k_cells=top_k_cells_fixed,
    max_new_tokens_sql=256,
)

evaluate_mmqa(
    json_path=mmqa3_test_path,
    output_prefix="mmqa3_test_topk_only",
    run_fn=run_pipeline_topk_only,
)



[EVAL] Loading dataset from: /content/drive/MyDrive/ai_intensive2/MMQA/mmqa3_test.json
[EVAL] #examples: 577
[EVAL] Cleared query_expander.cache for fresh token counting.


Evaluating mmqa3_test.json:   0%|                       | 0/577 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



[DEBUG] token_usage sample:
{'pipeline_qexp': {'input': 181, 'output': 91, 'total': 272},
 'pipeline_sql_main': {'input': 484, 'output': 70, 'total': 554},
 'pipeline_sql_selfcheck': {'input': 411, 'output': 70, 'total': 481}}


Evaluating mmqa3_test.json: 100%|███████████| 577/577 [1:49:19<00:00, 11.37s/it]


[SAVE] Per-example results saved to: /content/drive/MyDrive/ai_intensive2/MMQA/mmqa3_test_topk_only_results.json
[SAVE] Summary report saved to      : /content/drive/MyDrive/ai_intensive2/MMQA/mmqa3_test_topk_only_report.json
